<a href="https://colab.research.google.com/github/garima24112000/Scaling-Retrieval-System/blob/main/Step2%263_Retrieval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install numpy==1.26.4 pandas==2.1.4 pyarrow==14.0.2 tqdm pyyaml -q
!pip install sentence-transformers -q
!pip install faiss-cpu>=1.12.0 -q
!pip install pyspark==3.5.1 -q
!pip install chromadb==0.4.24 -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 105.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 117.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.0/38.0 MB 56.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.1.4 which is incompatible.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
plotnine 0.14.5 requires pandas>=2.2.0, but you have pandas 2.1.4 which is incompatible.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
pytensor 2.35.1 requires numpy>=2.0, but you have numpy 1.26

In [ ]:
import os
os.kill(os.getpid(), 9)

In [ ]:
import numpy, pandas, pyarrow, faiss, pyspark
from sentence_transformers import SentenceTransformer
import chromadb

# **Mount Drive + set ONE project root**

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Define ONE project root for everything (edit if your folder is different)
ROOT = "/content/drive/MyDrive/AMS 560 PROJECT"

# Move into your project root
%cd $ROOT

print("Project root set to:", ROOT)


Mounted at /content/drive
/content/drive/MyDrive/AMS 560 PROJECT
Project root set to: /content/drive/MyDrive/AMS 560 PROJECT


# **Install required dependencies**

In [ ]:
# Install only what's required for retrieval (no embedding, no Spark here)
!pip install -q sentence-transformers faiss-cpu pandas pyarrow



# **Create necessary folders**

In [ ]:
# Create config, results, and query folders if they don't already exist
import os
os.makedirs(os.path.join(ROOT, "configs/index"), exist_ok=True)
os.makedirs(os.path.join(ROOT, "results"), exist_ok=True)
os.makedirs(os.path.join(ROOT, "data/queries"), exist_ok=True)

print("Folders ready under", ROOT)



Folders ready under /content/drive/MyDrive/AMS 560 PROJECT


# **Write config.json (points to saved FAISS + meta)**

In [ ]:
# Build a config dictionary using ROOT so paths never break
import os, json

cfg = {
  "active_index": "multi_default",
  "indices": {
    "multi_default": {
      "faiss_path": os.path.join(ROOT, "indices/multi_default/multi_hnsw.faiss"),
      "meta_path":  os.path.join(ROOT, "data/embeddings/multi_default/multi_meta.parquet"),
      "model_name": "BAAI/bge-small-en-v1.5",
      "metric": "ip",
      "normalize_query": True
    },
    "single_wiki": {
      # 👇 edit these if your wiki index/meta are elsewhere
      "faiss_path": os.path.join(ROOT, "indices/wikipedia/wiki_hnsw_M32.faiss"),
      "meta_path":  os.path.join(ROOT, "data/embeddings/wikipedia/wiki_meta.parquet"),
      "model_name": "BAAI/bge-small-en-v1.5",
      "metric": "ip",
      "normalize_query": True
    }
  }
}

config_path = os.path.join(ROOT, "configs/index/config.json")
with open(config_path, "w") as f:
    json.dump(cfg, f, indent=2)

print(" Wrote config to:", config_path)
print(json.dumps(cfg, indent=2))


 Wrote config to: /content/drive/MyDrive/AMS 560 PROJECT/configs/index/config.json
{
  "active_index": "multi_default",
  "indices": {
    "multi_default": {
      "faiss_path": "/content/drive/MyDrive/AMS 560 PROJECT/indices/multi_default/multi_hnsw.faiss",
      "meta_path": "/content/drive/MyDrive/AMS 560 PROJECT/data/embeddings/multi_default/multi_meta.parquet",
      "model_name": "BAAI/bge-small-en-v1.5",
      "metric": "ip",
      "normalize_query": true
    },
    "single_wiki": {
      "faiss_path": "/content/drive/MyDrive/AMS 560 PROJECT/indices/wikipedia/wiki_hnsw_M32.faiss",
      "meta_path": "/content/drive/MyDrive/AMS 560 PROJECT/data/embeddings/wikipedia/wiki_meta.parquet",
      "model_name": "BAAI/bge-small-en-v1.5",
      "metric": "ip",
      "normalize_query": true
    }
  }
}


In [ ]:
!cat configs/index/config.json


{
  "active_index": "multi_default",
  "indices": {
    "multi_default": {
      "faiss_path": "/content/drive/MyDrive/AMS 560 PROJECT/indices/multi_default/multi_hnsw.faiss",
      "meta_path": "/content/drive/MyDrive/AMS 560 PROJECT/data/embeddings/multi_default/multi_meta.parquet",
      "model_name": "BAAI/bge-small-en-v1.5",
      "metric": "ip",
      "normalize_query": true
    },
    "single_wiki": {
      "faiss_path": "/content/drive/MyDrive/AMS 560 PROJECT/indices/wikipedia/wiki_hnsw_M32.faiss",
      "meta_path": "/content/drive/MyDrive/AMS 560 PROJECT/data/embeddings/wikipedia/wiki_meta.parquet",
      "model_name": "BAAI/bge-small-en-v1.5",
      "metric": "ip",
      "normalize_query": true
    }
  }
}

# **Write retrieval_runner.py**

In [ ]:
# This script loads model + FAISS + metadata and runs queries.
# It does NOT re-embed or rebuild indices.
%%writefile retrieval_runner.py
import os, json, argparse, time
import numpy as np, pandas as pd
try:
    import faiss
except Exception:
    import faiss_cpu as faiss
from sentence_transformers import SentenceTransformer

class RetrievalRunner:
    """Loads FAISS + metadata and returns search results."""
    def __init__(self, config_path: str, index_key: str | None = None):
        # Load config
        with open(config_path, "r") as f:
            cfg = json.load(f)
        index_key = index_key or cfg.get("active_index")
        if not index_key or index_key not in cfg["indices"]:
            raise ValueError(f"Index key '{index_key}' missing or not found in config.")
        self.icfg = cfg["indices"][index_key]

        # 1) Load FAISS index
        if not os.path.exists(self.icfg["faiss_path"]):
            raise FileNotFoundError(f"FAISS index not found: {self.icfg['faiss_path']}")
        self.index = faiss.read_index(self.icfg["faiss_path"])
        print(f"Loaded FAISS index: {self.icfg['faiss_path']}")

        # 2) Load metadata
        if not os.path.exists(self.icfg["meta_path"]):
            raise FileNotFoundError(f"Meta parquet not found: {self.icfg['meta_path']}")
        self.meta  = pd.read_parquet(self.icfg["meta_path"])
        print(f"Loaded metadata rows: {len(self.meta)}")

        # 3) Load query encoder
        self.model = SentenceTransformer(self.icfg.get("model_name", "BAAI/bge-small-en-v1.5"))
        print(f"Loaded model: {self.icfg.get('model_name')}")

        # Search settings
        self.normalize_query = bool(self.icfg.get("normalize_query", True))
        try:
            faiss.ParameterSpace().set_index_parameter(self.index, "efSearch", 64)
        except Exception:
            pass

    def _encode(self, q: str) -> np.ndarray:
        """Encode text query to a vector."""
        if not q or not q.strip():
            raise ValueError("Empty query.")
        vec = self.model.encode([q], normalize_embeddings=self.normalize_query).astype("float32")
        return vec

    def search(self, query: str, top_k: int = 5, filter_source: str | None = None) -> list[dict]:
        """
        Returns list of dicts:
        rank, score, id, source, title, url, text, chunk_id
        """
        qv = self._encode(query)
        overfetch = top_k * 4 if filter_source else top_k
        D, I = self.index.search(qv, overfetch)
        results, taken = [], 0

        for idx, score in zip(I[0], D[0]):
            if idx < 0:
                continue
            row = self.meta.iloc[idx]
            source = row.get("domain", "") or row.get("source", "")
            if filter_source and source != filter_source:
                continue
            results.append({
                "rank": taken + 1,
                "score": float(score),
                "id": row.get("id", ""),
                "source": source,
                "title": row.get("title", ""),
                "url": row.get("url", ""),
                "text": row.get("text", row.get("chunk_text", "")),
                "chunk_id": row.get("chunk_id", None)
            })
            taken += 1
            if taken >= top_k:
                break
        return results

def main():
    ap = argparse.ArgumentParser()
    ap.add_argument("--config", required=True, help="Path to configs/index/config.json")
    ap.add_argument("--index", default=None, help="Index key to use (overrides active_index)")
    ap.add_argument("--query", default=None, help="Single query string")
    ap.add_argument("--qfile", default=None, help="Optional newline-delimited queries file")
    ap.add_argument("--top_k", type=int, default=5)
    ap.add_argument("--filter_source", default=None, help="e.g., wikipedia | pubmed | stackexchange")
    ap.add_argument("--out_csv", default=None, help="If set, write results to CSV at this path")
    args = ap.parse_args()

    rr = RetrievalRunner(args.config, args.index)

    # collect queries
    queries = []
    if args.query:
        queries.append(args.query)
    if args.qfile and os.path.exists(args.qfile):
        with open(args.qfile) as f:
            queries += [ln.strip() for ln in f if ln.strip()]
    if not queries:
        raise SystemExit("Provide --query or --qfile")

    # run
    t0 = time.time()
    rows = []
    for q in queries:
        hits = rr.search(q, top_k=args.top_k, filter_source=args.filter_source)
        for h in hits:
            rows.append({"query": q, **h})
    dt = time.time() - t0
    print(f"🕒 Ran {len(queries)} queries in {dt:.3f}s; avg {dt/len(queries):.3f}s/query")

    # output
    if args.out_csv:
        os.makedirs(os.path.dirname(args.out_csv), exist_ok=True)
        pd.DataFrame(rows).to_csv(args.out_csv, index=False)
        print("💾 Saved:", args.out_csv)
    else:
        for r in rows:
            print(f"\nQ: {r['query']}")
            print(f"{r['rank']}. [{r['source']}] {r['title']} (score={r['score']:.4f})")
            if r.get("url"):
                print(r["url"])

if __name__ == "__main__":
    main()


Overwriting retrieval_runner.py


# **Quick sanity check: show config and file presence**

In [ ]:
import os, json
config_path = os.path.join(ROOT, "configs/index/config.json")
print("Config exists:", os.path.exists(config_path))
print("Config path:", config_path)
with open(config_path) as f:
    print(f.read()[:400], "...\n")

# Also confirm index/meta exist (prints True/False)
import json
cfg = json.load(open(config_path))
icfg = cfg["indices"][cfg["active_index"]]
print("Index exists:", os.path.exists(icfg["faiss_path"]), "→", icfg["faiss_path"])
print("Meta exists :", os.path.exists(icfg["meta_path"]), "→", icfg["meta_path"])


Config exists: True
Config path: /content/drive/MyDrive/AMS 560 PROJECT/configs/index/config.json
{
  "active_index": "multi_default",
  "indices": {
    "multi_default": {
      "faiss_path": "/content/drive/MyDrive/AMS 560 PROJECT/indices/multi_default/multi_hnsw.faiss",
      "meta_path": "/content/drive/MyDrive/AMS 560 PROJECT/data/embeddings/multi_default/multi_meta.parquet",
      "model_name": "BAAI/bge-small-en-v1.5",
      "metric": "ip",
      "normalize_query": true
    },
    "sing ...

Index exists: True → /content/drive/MyDrive/AMS 560 PROJECT/indices/multi_default/multi_hnsw.faiss
Meta exists : True → /content/drive/MyDrive/AMS 560 PROJECT/data/embeddings/multi_default/multi_meta.parquet


# **Verify artifacts + alignment (quick guardrails)**

In [ ]:
# Alignment check: FAISS vectors == meta rows
import json, os, pandas as pd
import faiss

config_path = os.path.join(ROOT, "configs/index/config.json")
cfg = json.load(open(config_path))
icfg = cfg["indices"][cfg["active_index"]]

index = faiss.read_index(icfg["faiss_path"])
meta  = pd.read_parquet(icfg["meta_path"])

print("FAISS ntotal:", index.ntotal)
print("META rows  :", len(meta))
assert index.ntotal == len(meta), "Mismatch! Index and meta are not row-aligned."
print("Alignment OK")


FAISS ntotal: 79164
META rows  : 79164
Alignment OK


# **Run merged (no-filter) tests that hit all 3 sources**

In [ ]:
# Write a balanced 12-query set likely to pull from wikipedia, pubmed, stackexchange
import os, textwrap
allq_path = os.path.join(ROOT, "data/queries/all_sources_eval.txt")
qtext = textwrap.dedent("""\
metformin mechanism of action
adverse effects of statins
type 2 diabetes lifestyle interventions evidence
randomized controlled trial definition
How to fix ModuleNotFoundError in Python
SQL window function ROW_NUMBER example
difference between git merge and rebase
What is a memory leak and how to detect it
Explain PageRank algorithm
definition of entropy in information theory
history of backpropagation
What is overfitting vs underfitting
""")
open(allq_path, "w").write(qtext)
print(" Wrote:", allq_path)


 Wrote: /content/drive/MyDrive/AMS 560 PROJECT/data/queries/all_sources_eval.txt


In [ ]:
# Run retrieval on merged index, save top-10 per query
all_csv = os.path.join(ROOT, "results/multi_default/all_sources_top10.csv")
!python retrieval_runner.py \
  --config "configs/index/config.json" \
  --index multi_default \
  --qfile "data/queries/all_sources_eval.txt" \
  --top_k 10 \
  --out_csv "{all_csv}"
print(" Saved:", all_csv)


2025-12-04 16:51:43.710595: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764867103.731710    1596 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764867103.738214    1596 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1764867103.754378    1596 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764867103.754405    1596 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764867103.754408    1596 computation_placer.cc:177] computation placer alr

#quick coverage report (proof all three domains appear)

In [ ]:
import pandas as pd, os
all_csv = os.path.join(ROOT, "results/multi_default/all_sources_top10.csv")
df = pd.read_csv(all_csv)

print("Rows:", len(df))
print("\nHits by source:")
print(df["source"].value_counts(dropna=False))

print("\nFirst hit per query:")
firsts = (df.sort_values(["query","rank"])
            .groupby("query")
            .first()[["source","title","url"]])
display(firsts.head(20))


Rows: 120

Hits by source:
source
stackexchange    46
wikipedia        42
pubmed           32
Name: count, dtype: int64

First hit per query:


,source,title,url
query,,,
Explain PageRank algorithm,wikipedia,Search engine,https://en.wikipedia.org/wiki/Search%20engine
How to fix ModuleNotFoundError in Python,stackexchange,Is my tkcalendar clashing with ttkbootstrap?,https://stackoverflow.com/questions/79751755/i...
SQL window function ROW_NUMBER example,stackexchange,How to pass fieldnames containing # to createM...,https://stackoverflow.com/questions/79768985/h...
What is a memory leak and how to detect it,wikipedia,Buffer overflow,https://en.wikipedia.org/wiki/Buffer%20overflow
What is overfitting vs underfitting,stackexchange,Why does my logistic curve_fit prediction retu...,https://stackoverflow.com/questions/79748043/w...
adverse effects of statins,pubmed,Updated Evidence on the Protective Role of Sta...,https://pubmed.ncbi.nlm.nih.gov/41170235/
definition of entropy in information theory,wikipedia,Kolmogorov complexity,https://en.wikipedia.org/wiki/Kolmogorov%20com...
difference between git merge and rebase,stackexchange,Is there any advantage to updating a branch th...,https://stackoverflow.com/questions/79724991/i...
history of backpropagation,wikipedia,Artificial intelligence,https://en.wikipedia.org/wiki/Artificial%20int...


#single-query demos

In [ ]:
# Biomedical-ish → likely PubMed in mix
!python retrieval_runner.py --config "configs/index/config.json" --index multi_default \
  --query "GLP-1 receptor agonists weight loss evidence" --top_k 8

# Coding/how-to → likely StackExchange in mix
!python retrieval_runner.py --config "configs/index/config.json" --index multi_default \
  --query "Best practice for SQL indexing composite keys" --top_k 8

# General CS/knowledge → likely Wikipedia in mix
!python retrieval_runner.py --config "configs/index/config.json" --index multi_default \
  --query "Explain PageRank algorithm in simple terms" --top_k 8


2025-12-04 16:52:07.227708: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764867127.248575    1808 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764867127.254954    1808 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1764867127.270564    1808 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764867127.270594    1808 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764867127.270599    1808 computation_placer.cc:177] computation placer alr

#timing log (repeatable benchmark output)

In [ ]:
# Re-run the 12-query set and also write a small timing sidecar
import time, subprocess, os, json
start = time.time()
all_csv = os.path.join(ROOT, "results/multi_default/all_sources_top10.csv")
cmd = f'python retrieval_runner.py --config "configs/index/config.json" --index multi_default --qfile "data/queries/all_sources_eval.txt" --top_k 10 --out_csv "{all_csv}"'
ret = subprocess.run(cmd, shell=True, capture_output=True, text=True)
print(ret.stdout)
elapsed = time.time()-start

timing_path = os.path.join(ROOT, "results/multi_default/all_sources_timing.json")
json.dump({"elapsed_seconds": elapsed, "queries": 12, "avg_per_query": elapsed/12}, open(timing_path,"w"), indent=2)
print("Timing saved:", timing_path)


Loaded FAISS index: /content/drive/MyDrive/AMS 560 PROJECT/indices/multi_default/multi_hnsw.faiss
Loaded metadata rows: 79164
Loaded model: BAAI/bge-small-en-v1.5
🕒 Ran 12 queries in 0.406s; avg 0.034s/query
💾 Saved: /content/drive/MyDrive/AMS 560 PROJECT/results/multi_default/all_sources_top10.csv

Timing saved: /content/drive/MyDrive/AMS 560 PROJECT/results/multi_default/all_sources_timing.json


# efSearch configurable

In [ ]:
# Patch: if you want to try a faster/more accurate HNSW search, bump efSearch to 128 in code.
# (You can also add "efSearch": 128 inside config.json under each index to formalize it.)
from json import load, dump
cfg_path = os.path.join(ROOT, "configs/index/config.json")
cfg = load(open(cfg_path))
for k,v in cfg["indices"].items():
    v["efSearch"] = 128
dump(cfg, open(cfg_path, "w"), indent=2)
print("Added efSearch=128 to config")

# Update retrieval_runner.py (one-line enhancement): use efSearch if present
patch = """
        try:
            from faiss import ParameterSpace
            ps = ParameterSpace()
            ps.set_index_parameter(self.index, "efSearch", int(self.icfg.get("efSearch", 64)))
        except Exception:
            pass
"""
print("Add this block (replacing the old efSearch try/except) inside RetrievalRunner.__init__ if you want dynamic efSearch.")


Added efSearch=128 to config
Add this block (replacing the old efSearch try/except) inside RetrievalRunner.__init__ if you want dynamic efSearch.


# Step-2 Retrieval Runner (Kashish README)
**What this does**
- Loads merged FAISS + merged meta (wikipedia+pubmed+stackexchange).
- Encodes queries with BAAI/bge-small-en-v1.5.
- Returns top-k results with title/url/source/text.
- Saves CSV results for Team-3 benchmarking.

**How to run**
```bash
python retrieval_runner.py \\
  --config "configs/index/config.json" \\
  --index multi_default \\
  --qfile "data/queries/all_sources_eval.txt" \\
  --top_k 10 \\
  --out_csv "results/multi_default/all_sources_top10.csv"


# Retrieval System Extension

## **Step 3.1 — Config loader + Backend interface**

What this adds?

pipeline/config.py → loads JSON or YAML, validates schema, and returns a normalized config for a chosen index (single or sharded).

pipeline/backends/base.py → defines a pluggable VectorBackend interface so FAISS/Chroma/Spark-sharded implementations can slot in later.

Why?

Lets you switch backend / model / index (single vs shards) just by editing config

Create folders (once)

In [ ]:
# Make pipeline folders (safe if they already exist)
import os
os.makedirs("pipeline/backends", exist_ok=True)
os.makedirs("results/logs", exist_ok=True)


In [ ]:
import os
# Hide TensorFlow’s C++ logs
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
# Tell Transformers to NOT import TensorFlow/Flax at all
os.environ["TRANSFORMERS_NO_TF"] = "1"
os.environ["TRANSFORMERS_NO_FLAX"] = "1"

Write pipeline/config.py

In [ ]:
%%writefile pipeline/config.py
"""
Config loader for retrieval pipeline (scaling-ready).

- Supports JSON (.json) and YAML (.yml/.yaml)
- Validates index blocks (single-index or sharded)
- Normalizes keys so backends see a consistent shape
- Adds schema versioning and metric/normalization semantics
- Supports remote paths (s3://, gs://, hdfs://) and optional path checks
"""

from __future__ import annotations
import json
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Tuple
import os

try:
    import yaml  # type: ignore
except Exception as e:
    raise ImportError(
        "pyyaml is required. Install with: pip install pyyaml"
    ) from e


class ConfigError(Exception):
    """Raised when configuration is missing/invalid."""


# Accepted config schema versions (simple gate for now)
_ACCEPTED_CONFIG_VERSIONS = {"1"}


@dataclass
class NormalizedIndexConfig:
    # core knobs
    backend: str
    model_name: str
    normalize_query: bool
    metric: str
    efSearch: int

    # union of single vs sharded
    faiss_path: Optional[str] = None
    meta_path: Optional[str] = None
    index_paths: Optional[List[str]] = None
    meta_paths: Optional[List[str]] = None

    # semantics (for cross-backend parity & validation)
    index_metric: Optional[str] = None  # 'ip' | 'l2' | 'cosine'
    vector_normed: Optional[bool] = None  # True if stored vectors are unit-normalized

    # passthrough
    extras: Dict[str, Any] = field(default_factory=dict)

    # loader context
    index_key: Optional[str] = None
    config_version: Optional[str] = None
    skip_path_checks: bool = False


def _load_raw_config(path: str) -> Dict[str, Any]:
    if not os.path.exists(path):
        raise ConfigError(f"Config file not found: {path}")

    _, ext = os.path.splitext(path.lower())

    try:
        with open(path, "r", encoding="utf-8") as f:
            if ext == ".json":
                return json.load(f)
            if ext in (".yml", ".yaml"):
                return yaml.safe_load(f)

            # unknown extension → try JSON then YAML
            try:
                f.seek(0)
                return json.load(f)
            except Exception:
                f.seek(0)
                return yaml.safe_load(f)

    except Exception as e:
        raise ConfigError(f"Failed to parse config {path}: {e}") from e


def _require(d: Dict[str, Any], key: str, ctx: str) -> Any:
    if key not in d:
        raise ConfigError(f"Missing required key '{key}' in {ctx}")
    return d[key]


def _as_bool(v: Any, default: bool) -> bool:
    if v is None:
        return default
    if isinstance(v, bool):
        return v
    if isinstance(v, (int, float)):
        return bool(v)
    if isinstance(v, str):
        s = v.strip().lower()
        if s in {"1", "true", "yes", "y", "on"}:
            return True
        if s in {"0", "false", "no", "n", "off"}:
            return False
    raise ConfigError(f"Expected boolean-like value, got: {v!r}")


def _as_int(v: Any, default: int) -> int:
    if v is None:
        return default
    if isinstance(v, int):
        return v
    if isinstance(v, float):
        return int(v)
    if isinstance(v, str):
        try:
            return int(float(v.strip()))
        except Exception:
            pass
    raise ConfigError(f"Expected integer-like value, got: {v!r}")


def _is_remote_path(p: str) -> bool:
    p = (p or "").lower()
    return p.startswith("s3://") or p.startswith("gs://") or p.startswith("hdfs://")


def _validate_paths_exist(paths: List[str], ctx: str, *, skip_checks: bool) -> None:
    if skip_checks:
        return
    missing = [p for p in paths if (not _is_remote_path(p)) and (not os.path.exists(p))]
    if missing:
        raise ConfigError(f"{ctx}: missing files:\n - " + "\n - ".join(missing))


def _normalize_index_block(
    block: Dict[str, Any],
    ctx: str,
    index_key: str,
    top_level_skip_checks: bool,
) -> NormalizedIndexConfig:

    # knobs
    backend = (block.get("backend") or "faiss").lower()
    model_name = block.get("model_name", "BAAI/bge-small-en-v1.5")
    metric = (block.get("metric") or "ip").lower()
    efSearch = _as_int(block.get("efSearch"), 64)
    normalize_query = _as_bool(block.get("normalize_query"), True)

    # semantics
    index_metric = (block.get("index_metric") or metric or "").lower() or None

    vector_normed = block.get("vector_normed")
    if vector_normed is not None:
        vector_normed = _as_bool(vector_normed, False)

    # single vs sharded
    faiss_path = block.get("faiss_path")
    meta_path = block.get("meta_path")
    index_paths = block.get("index_paths")
    meta_paths = block.get("meta_paths")

    single_ok = bool(faiss_path and meta_path)
    shard_ok = bool(index_paths and meta_paths)

    if single_ok and shard_ok:
        raise ConfigError(
            f"{ctx}: Provide EITHER single (faiss_path/meta_path) OR sharded "
            f"(index_paths/meta_paths), not both."
        )

    if not (single_ok or shard_ok):
        raise ConfigError(
            f"{ctx}: Must provide single (faiss_path+meta_path) OR sharded "
            f"(index_paths+meta_paths)."
        )

    # skip checks: block-level overrides top-level
    skip_path_checks = _as_bool(block.get("skip_path_checks"), top_level_skip_checks)

    # path validation
    if single_ok:
        _validate_paths_exist(
            [faiss_path, meta_path], f"{ctx} (single)", skip_checks=skip_path_checks
        )
    else:
        if not isinstance(index_paths, list) or not isinstance(meta_paths, list):
            raise ConfigError(
                f"{ctx}: index_paths/meta_paths must be lists for sharded mode."
            )
        if len(index_paths) != len(meta_paths):
            raise ConfigError(
                f"{ctx}: index_paths and meta_paths length mismatch "
                f"({len(index_paths)} vs {len(meta_paths)})."
            )

        _validate_paths_exist(
            index_paths, f"{ctx} index_paths", skip_checks=skip_path_checks
        )
        _validate_paths_exist(
            meta_paths, f"{ctx} meta_paths", skip_checks=skip_path_checks
        )

    # extras
    extras = {
        k: v
        for k, v in block.items()
        if k
        not in {
            "backend",
            "model_name",
            "normalize_query",
            "metric",
            "efSearch",
            "faiss_path",
            "meta_path",
            "index_paths",
            "meta_paths",
            "index_metric",
            "vector_normed",
            "skip_path_checks",
        }
    }

    return NormalizedIndexConfig(
        backend=backend,
        model_name=model_name,
        normalize_query=normalize_query,
        metric=metric,
        efSearch=efSearch,
        faiss_path=faiss_path,
        meta_path=meta_path,
        index_paths=index_paths,
        meta_paths=meta_paths,
        index_metric=index_metric,
        vector_normed=vector_normed,
        extras=extras,
        index_key=index_key,
        config_version=None,  # filled by loader
        skip_path_checks=skip_path_checks,
    )


def load_config(
    path: str, index_key: Optional[str] = None
) -> Tuple[Dict[str, Any], NormalizedIndexConfig]:
    """
    Returns (raw_config_dict, normalized_index_cfg) for the chosen index key.

    - If index_key is None, uses raw['active_index'].
    - Validates the selected index block.
    - Enforces schema version and remote-path/skip-check behavior.
    """

    raw = _load_raw_config(path)
    if not isinstance(raw, dict):
        raise ConfigError("Top-level config must be an object/dict.")

    # schema version
    cfg_version = str(raw.get("config_version", "1")).strip()
    if cfg_version not in _ACCEPTED_CONFIG_VERSIONS:
        raise ConfigError(
            f"Unsupported config_version='{cfg_version}'. "
            f"Accepted versions: {_ACCEPTED_CONFIG_VERSIONS}"
        )

    indices = _require(raw, "indices", "root")
    if not isinstance(indices, dict) or not indices:
        raise ConfigError("'indices' must be a non-empty object.")

    if index_key is None:
        index_key = raw.get("active_index")
        if not index_key:
            raise ConfigError("Provide index_key or set 'active_index' in config.")

    if index_key not in indices:
        raise ConfigError(f"Index key '{index_key}' not found in 'indices'.")

    block = indices[index_key]
    if not isinstance(block, dict):
        raise ConfigError(f"Index '{index_key}' must be an object.")

    top_level_skip_checks = _as_bool(raw.get("skip_path_checks"), False)

    nic = _normalize_index_block(
        block, f"index '{index_key}'", index_key, top_level_skip_checks
    )
    nic.config_version = cfg_version

    # warn on metric mismatch
    if nic.index_metric and nic.metric and nic.index_metric != nic.metric:
        import warnings

        warnings.warn(
            f"Requested metric '{nic.metric}' differs from index_metric '{nic.index_metric}'. "
            "Ensure query normalization / FAISS setup matches."
        )

    return raw, nic


Overwriting pipeline/config.py


Write pipeline/backends/base.py

In [ ]:
%%writefile pipeline/backends/base.py
"""
Backend interface for retrieval pipeline.

Concrete backends must implement:
- load(cfg): prepare model, index/connection, metadata
- encode(text_or_list): return float32 numpy array (n, d)
- search(qvec, top_k): return list[dict] with keys:
    rank, score, id, source, title, url, text, chunk_id
- close(): free resources
"""

from __future__ import annotations
from typing import Any, Dict, List, Protocol, runtime_checkable


Result = Dict[str, Any]


@runtime_checkable
class VectorBackend(Protocol):
    def load(self, cfg: Any) -> None:  # cfg is NormalizedIndexConfig or similar
        ...

    def encode(self, text_or_list: Any) -> Any:
        """
        Accepts a string or list[str] and returns np.ndarray[float32] of shape (n, d).
        Implementations should respect 'normalize_query' if applicable.
        """
        ...

    def search(self, qvec: Any, top_k: int = 5, filter_source: str | None = None) -> List[Result]:
        """
        Accepts a single query vector (1, d) and returns a list of result dicts:
        {
          'rank': int,
          'score': float,
          'id': str,
          'source': str,   # domain
          'title': str,
          'url': str,
          'text': str,     # chunk_text or raw text
          'chunk_id': int|None
        }
        """
        ...

    def close(self) -> None:
        ...


Overwriting pipeline/backends/base.py


Quick sanity tests (run these)

In [ ]:
if __name__ == "__main__":
    from pipeline.config import load_config, NormalizedIndexConfig, ConfigError
    print("Imports OK")
    raw, nic = load_config("configs/index/config.json", index_key=None)
    print("Active index normalized:", nic)
    try:
        _ = load_config("configs/index/config.json", index_key="does_not_exist")
    except ConfigError as e:
        print("Expected error (bad index key):", e)

Imports OK
Active index normalized: NormalizedIndexConfig(backend='faiss', model_name='BAAI/bge-small-en-v1.5', normalize_query=True, metric='ip', efSearch=128, faiss_path='/content/drive/MyDrive/AMS 560 PROJECT/indices/multi_default/multi_hnsw.faiss', meta_path='/content/drive/MyDrive/AMS 560 PROJECT/data/embeddings/multi_default/multi_meta.parquet', index_paths=None, meta_paths=None, index_metric='ip', vector_normed=None, extras={}, index_key='multi_default', config_version='1', skip_path_checks=False)
Expected error (bad index key): Index key 'does_not_exist' not found in 'indices'.


In [ ]:
from pipeline.config import load_config, ConfigError
try:
    # Temporarily point to a non-existent file in a copy of the config and load it
    import json, shutil
    shutil.copy("configs/index/config.json", "configs/index/config_tmp.json")
    cfg = json.load(open("configs/index/config_tmp.json"))
    cfg["indices"]["multi_default"]["faiss_path"] = "/not/a/real/path.faiss"
    json.dump(cfg, open("configs/index/config_tmp.json","w"), indent=2)
    _ = load_config("configs/index/config_tmp.json", index_key="multi_default")
except ConfigError as e:
    print("Path validation works →", e)


Path validation works → index 'multi_default' (single): missing files:
 - /not/a/real/path.faiss


## Step 3.2: FAISS backend + pipeline runner

What this adds?

pipeline/backends/faiss_backend.py — production-ready backend that:
*   loads single FAISS index or sharded indices (with heap-merge),
*   honors efSearch, normalize_query, metric,
*   supports filter_source (wikipedia/pubmed/stackexchange).

pipeline/runner.py — tiny façade:
*   load_pipeline(cfg_path, index_key=None, backend_override=None)
*   search_one(query, top_k, filter_source=None)
*   search_many(qfile, top_k, filter_source=None)

Why?

This replaces ad-hoc scripts with a pluggable, config-driven pipeline scaling can reuse for bigger stores.

Write pipeline/backends/faiss_backend.py

In [ ]:
%%writefile pipeline/backends/faiss_backend.py
from __future__ import annotations
import heapq, os
from typing import Any, Dict, List, Optional, Tuple

import numpy as np
import pandas as pd

try:
    import faiss  # faiss-cpu
except Exception:  # pragma: no cover
    import faiss_cpu as faiss  # fallback if aliased

from sentence_transformers import SentenceTransformer
from pipeline.backends.base import VectorBackend, Result
from pipeline.config import NormalizedIndexConfig


class FaissBackend(VectorBackend):
    """
    FAISS backend supporting:
      • Single index (faiss_path + meta_path)
      • Sharded indices (index_paths + meta_paths) with global top-k merge
    """

    def __init__(self) -> None:
        self.cfg: Optional[NormalizedIndexConfig] = None
        self.model: Optional[SentenceTransformer] = None
        self.normalize_query: bool = True
        self.metric: str = "ip"
        self._higher_is_better: bool = True  # derived from index/metric (ip/cosine=True, l2=False)

        # single-index
        self.index = None
        self.meta_df: Optional[pd.DataFrame] = None

        # sharded
        self.shard_indices: List[Any] = []
        self.shard_metas: List[pd.DataFrame] = []
        self.sharded: bool = False

    # ---------- helpers ----------

    @staticmethod
    def _set_efsearch(index, ef: int) -> None:
        try:
            ps = faiss.ParameterSpace()
            ps.set_index_parameter(index, "efSearch", int(ef))
        except Exception:
            # Some FAISS builds or index types don't support this param (e.g., non-HNSW)
            pass

    @staticmethod
    def _set_nprobe(index, nprobe: Optional[int]) -> None:
        if nprobe is None:
            return
        try:
            ps = faiss.ParameterSpace()
            ps.set_index_parameter(index, "nprobe", int(nprobe))
        except Exception:
            # Silently ignore if index is not IVF or FAISS build lacks ParameterSpace
            pass

    @staticmethod
    def _norm_metric_name(name: Optional[str]) -> str:
        if not name:
            return ""
        n = str(name).strip().lower()
        if n in {"cos", "cosine", "cos_sim"}:
            return "cosine"
        if n in {"ip", "inner", "inner_product"}:
            return "ip"
        if n in {"l2", "euclidean"}:
            return "l2"
        return n

    def _derive_metric_direction(self) -> None:
        """
        Decide whether larger score = better. We normalize so that the internal 'score'
        always follows higher-is-better across metrics.
        """
        # Prefer the index's actual metric if provided; otherwise requested metric.
        m = self._norm_metric_name(getattr(self.cfg, "index_metric", None) or self.metric)
        self._higher_is_better = (m in {"ip", "cosine"})

    @staticmethod
    def _src_of_row(row: pd.Series) -> str:
        """Return a normalized (lowercased) source/domain string; empty if missing."""
        s = row.get("domain", "") or row.get("source", "") or ""
        return str(s).strip().lower()

    def _encode_one(self, text: str) -> np.ndarray:
        assert self.model is not None
        v = self.model.encode([text], normalize_embeddings=self.normalize_query)
        return np.asarray(v, dtype="float32")

    def _row_to_result(self, row: pd.Series, score: float, rank: int) -> Result:
        # choose best available text field
        txt = (row.get("chunk_text") or row.get("text") or row.get("preview") or row.get("content") or "")
        cid = row.get("chunk_id")
        try:
            cid = int(cid) if cid is not None and cid == cid else None
        except Exception:
            cid = None
        uid = f"{row.get('id','')}::c{cid if cid is not None else -1}"

        return {
            "rank": rank,
            "score": float(score),
            "id": row.get("id", ""),
            "uid": uid,
            "source": row.get("domain", "") or row.get("source", ""),
            "title": row.get("title", ""),
            "url": row.get("url", ""),
            "text": txt,
            "chunk_id": cid,
        }

    # ---------- VectorBackend API ----------

    def load(self, cfg: NormalizedIndexConfig) -> None:
        self.cfg = cfg
        self.normalize_query = bool(cfg.normalize_query)
        self.metric = cfg.metric
        self._derive_metric_direction()

        # threading control (optional knob)
        threads = int(cfg.extras.get("faiss_num_threads", os.cpu_count() or 8))
        try:
            faiss.omp_set_num_threads(threads)
            print(f"[FAISS] Using {threads} threads")
        except Exception:
            pass

        # model
        self.model = SentenceTransformer(cfg.model_name)

        # optional IVF knob
        nprobe = cfg.extras.get("nprobe")

        # load single or sharded indices
        if cfg.faiss_path and cfg.meta_path:
            self.sharded = False
            self.index = faiss.read_index(cfg.faiss_path)
            self._set_efsearch(self.index, cfg.efSearch)
            self._set_nprobe(self.index, nprobe)
            self.meta_df = pd.read_parquet(cfg.meta_path).reset_index(drop=True)
        else:
            self.sharded = True
            for ipath, mpath in zip(cfg.index_paths or [], cfg.meta_paths or []):
                idx = faiss.read_index(ipath)
                self._set_efsearch(idx, cfg.efSearch)
                self._set_nprobe(idx, nprobe)
                self.shard_indices.append(idx)
                self.shard_metas.append(pd.read_parquet(mpath).reset_index(drop=True))

        # sanity checks
        if not self.sharded:
            if self.index.ntotal != len(self.meta_df):
                raise ValueError(f"FAISS ntotal={self.index.ntotal} != meta rows={len(self.meta_df)}")
        else:
            nvec = sum(ix.ntotal for ix in self.shard_indices)
            nmeta = sum(len(md) for md in self.shard_metas)
            if nvec != nmeta:
                raise ValueError(f"[sharded] vectors {nvec} != meta rows {nmeta}")

    def encode(self, text_or_list: Any) -> np.ndarray:
        if isinstance(text_or_list, str):
            return self._encode_one(text_or_list)
        elif isinstance(text_or_list, list):
            assert self.model is not None
            v = self.model.encode(text_or_list, normalize_embeddings=self.normalize_query)
            return np.asarray(v, dtype="float32")
        raise TypeError("encode() expects str or list[str]")

    # ---- search helpers ----
    def _search_single(self, qvec: np.ndarray, top_k: int, filter_source: Optional[str]) -> List[Result]:
        norm_filter = filter_source.strip().lower() if filter_source else None
        dedup_on = bool(getattr(self.cfg, "extras", {}).get("dedup_by_uid", False))
        seen_uids: set[str] = set()

        overfetch = top_k * 4 if norm_filter else top_k
        D, I = self.index.search(qvec, overfetch)
        out: List[Result] = []
        taken = 0
        for idx, raw in zip(I[0], D[0]):
            if idx < 0:
                continue
            row = self.meta_df.iloc[idx]
            if norm_filter:
                if self._src_of_row(row) != norm_filter:
                    continue
            # Normalize score so "higher is better" regardless of metric.
            score = float(raw if self._higher_is_better else -raw)
            res = self._row_to_result(row, score, taken + 1)

            if dedup_on:
                uid = res["uid"]
                if uid in seen_uids:
                    continue
                seen_uids.add(uid)

            taken += 1
            out.append(res)
            if taken >= top_k:
                break
        return out

    def _search_sharded(self, qvec: np.ndarray, top_k: int, filter_source: Optional[str]) -> List[Result]:
        norm_filter = filter_source.strip().lower() if filter_source else None
        dedup_on = bool(getattr(self.cfg, "extras", {}).get("dedup_by_uid", False))
        seen_uids: set[str] = set()

        alpha = int(self.cfg.extras.get("alpha", 3))
        per = max(top_k * alpha, top_k)
        heap: List[Tuple[float, int, int, str]] = []

        for s_id, idx in enumerate(self.shard_indices):
            D, I = idx.search(qvec, per)
            for raw, li in zip(D[0], I[0]):
                if li < 0:
                    continue
                # Normalize score; store as a max-heap via negative key.
                score = float(raw if self._higher_is_better else -raw)
                uid = str(li)
                heapq.heappush(heap, (-score, s_id, int(li), uid))

        out: List[Result] = []
        seen = 0
        while heap and seen < top_k:
            neg, s_id, li, _ = heapq.heappop(heap)
            score = -neg
            row = self.shard_metas[s_id].iloc[li]
            if norm_filter:
                if self._src_of_row(row) != norm_filter:
                    continue

            res = self._row_to_result(row, score, seen + 1)

            if dedup_on:
                uid = res["uid"]
                if uid in seen_uids:
                    continue
                seen_uids.add(uid)

            seen += 1
            out.append(res)
        return out

    def search(self, qvec: Any, top_k: int = 5, filter_source: str | None = None) -> List[Result]:
        if not isinstance(qvec, np.ndarray):
            raise TypeError("search() expects numpy array (1,d)")
        if qvec.ndim != 2 or qvec.shape[0] != 1:
            raise ValueError("search() expects shape (1,d)")
        return (
            self._search_sharded(qvec, top_k, filter_source)
            if self.sharded
            else self._search_single(qvec, top_k, filter_source)
        )

    def close(self) -> None:
        self.index = None
        self.shard_indices.clear()
        self.shard_metas.clear()
        self.meta_df = None
        self.model = None

Overwriting pipeline/backends/faiss_backend.py


Write pipeline/runner.py

In [ ]:
%%writefile pipeline/runner.py
from __future__ import annotations
from typing import Optional, List, Dict, Any
from pipeline.config import load_config, NormalizedIndexConfig
from pipeline.backends.base import VectorBackend
from pipeline.backends.faiss_backend import FaissBackend

# registry of backends
_REGISTRY = {
    "faiss": FaissBackend,
    # future: "chroma": ChromaBackend
}

class Pipeline:
    def __init__(self, backend: VectorBackend, nicfg: NormalizedIndexConfig):
        self.backend = backend
        self.nicfg = nicfg

    def search_one(self, query: str, top_k: int = 5, filter_source: str | None = None):
        qvec = self.backend.encode(query)
        return self.backend.search(qvec, top_k=top_k, filter_source=filter_source)

    def search_many(self, queries: List[str], top_k: int = 5, filter_source: str | None = None):
        qvecs = self.backend.encode(queries)
        out = {}
        for i, q in enumerate(queries):
            out[q] = self.backend.search(qvecs[i:i+1], top_k=top_k, filter_source=filter_source)
        return out

    def close(self):
        self.backend.close()


def load_pipeline(cfg_path: str, index_key: Optional[str] = None, backend_override: Optional[str] = None) -> Pipeline:
    raw, nic = load_config(cfg_path, index_key=index_key)
    backend_key = (backend_override or nic.backend or "faiss").lower()
    if backend_key not in _REGISTRY:
        raise ValueError(f"Unknown backend '{backend_key}'. Available: {list(_REGISTRY)}")
    backend = _REGISTRY[backend_key]()  # type: ignore
    backend.load(nic)
    return Pipeline(backend, nic)

Overwriting pipeline/runner.py


patch the sharded index block (indices.multi_sharded)

In [ ]:
import os, json
PROJECT_ROOT = "/content/drive/MyDrive/AMS 560 PROJECT"
CFG_PATH = f"{PROJECT_ROOT}/configs/index/config.json"
FAISS_PATH = f"{PROJECT_ROOT}/indices/multi_default/multi_hnsw.faiss"
META_PATH  = f"{PROJECT_ROOT}/data/embeddings/multi_default/multi_meta.parquet"

cfg = {
    "config_version": "1",
    "active_index": "multi_default",
    "indices": {
        "multi_default": {
            "backend": "faiss",
            "model_name": "BAAI/bge-small-en-v1.5",
            "normalize_query": True,
            "metric": "ip",
            "index_metric": "ip",
            "vector_normed": True,
            "efSearch": 128,
            "faiss_path": FAISS_PATH,
            "meta_path": META_PATH,
            "extras": {
                "alpha": 3,
                "dedup_by_uid": True,
                "faiss_num_threads": os.cpu_count() or 8,
                "text_fields": ["chunk_text", "text", "preview"],
                "nprobe": 32
            }
        }
    }
}
os.makedirs(os.path.dirname(CFG_PATH), exist_ok=True)
json.dump(cfg, open(CFG_PATH, "w"), indent=2)
print("✅ Wrote config:", CFG_PATH)

✅ Wrote config: /content/drive/MyDrive/AMS 560 PROJECT/configs/index/config.json


Hard-reload + quick sanity

In [ ]:
# 🔄 Hard-reload pipeline modules so the new FaissBackend is used
import sys, inspect

for m in list(sys.modules):
    if m.startswith("pipeline."):
        del sys.modules[m]

from pipeline.runner import load_pipeline
from pipeline.backends.faiss_backend import FaissBackend

print("row_to_result signature:", inspect.signature(FaissBackend._row_to_result))
print("Source snippet:\n", "\n".join(inspect.getsource(FaissBackend._row_to_result).splitlines()[:6]))

row_to_result signature: (self, row: 'pd.Series', score: 'float', rank: 'int') -> 'Result'
Source snippet:
     def _row_to_result(self, row: pd.Series, score: float, rank: int) -> Result:
        # choose best available text field
        txt = (row.get("chunk_text") or row.get("text") or row.get("preview") or row.get("content") or "")
        cid = row.get("chunk_id")
        try:
            cid = int(cid) if cid is not None and cid == cid else None


Smoke test

In [ ]:
import sys
for m in list(sys.modules):
    if m.startswith("pipeline."):
        del sys.modules[m]

from pipeline.runner import load_pipeline

P = load_pipeline("configs/index/config.json")  # uses active_index
hits = P.search_one("Explain PageRank algorithm", top_k=5)
print(f"Got {len(hits)} hits. First title:", hits[0]["title"], "| source:", hits[0]["source"])

hits_pubmed = P.search_one("metformin mechanism of action", top_k=5, filter_source="pubmed")
print("pubmed first hit:", hits_pubmed[0]["source"], hits_pubmed[0]["title"])

P.close()

[FAISS] Using 12 threads


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Got 5 hits. First title: Search engine | source: wikipedia
pubmed first hit: pubmed The beneficial effects of empagliflozin-metformin combination on cardiovascular risk factors in type 2 diabetes mellitus patients.


## Step 3.3 — Sharded FAISS + Global Top-K Merge (“Local Fan-Out”)

Objective

Splits the multi-domain FAISS datastore into multiple shards (e.g., 8), then extend the pipeline so that a single query is:

broadcast to each shard,

searched locally (with efSearch ≈ 64 - 128),

merged globally using a heap-based top-K aggregator.

**Shard the Existing Multi Index**

Script: scripts/shard_index.py

This takes:


```
data/embeddings/multi_default/multi_meta.parquet
data/embeddings/multi_default/multi_vectors.npy
```
and produces:

```
data/embeddings/multi_default_sharded/meta_shard_000.parquet …
indices/multi_default_sharded/shard_000.faiss …
indices/multi_default_sharded/manifest.json
```



In [ ]:
# Make folders
!mkdir -p scripts
!mkdir -p "/content/drive/MyDrive/AMS 560 PROJECT/indices/multi_default_sharded"
!mkdir -p "/content/drive/MyDrive/AMS 560 PROJECT/data/embeddings/multi_default_sharded"

In [ ]:
%%writefile scripts/shard_index.py
"""
Shard the merged FAISS dataset into N shards with metric parity + manifest.

Usage (typical):
    python scripts/shard_index.py --n_shards 8 \
        --metric ip --normalize 1 --M 32 --efC 200 --split hash --seed 42 \
        --save_vectors 0

Inputs (fixed paths under project_root):
    data/embeddings/multi_default/multi_meta.parquet
    data/embeddings/multi_default/multi_vectors.npy

Outputs:
    data/embeddings/multi_default_sharded/meta_shard_000.parquet ...
    indices/multi_default_sharded/shard_000.faiss ...
    indices/multi_default_sharded/manifest.json
    # (optional, only if --save_vectors 1)
    data/embeddings/multi_default_sharded/vecs_shard_000.npy ...
"""

import os, json, argparse, time, hashlib, sys
from datetime import datetime, timezone

import numpy as np
import pandas as pd
from tqdm import tqdm
import faiss


project_root = "/content/drive/MyDrive/AMS 560 PROJECT"
EMB_DIR  = f"{project_root}/data/embeddings/multi_default"
OUT_EMB  = f"{project_root}/data/embeddings/multi_default_sharded"
OUT_IDX  = f"{project_root}/indices/multi_default_sharded"
os.makedirs(OUT_EMB, exist_ok=True)
os.makedirs(OUT_IDX, exist_ok=True)


def _normalize_rows(x: np.ndarray) -> np.ndarray:
    norms = np.linalg.norm(x, axis=1, keepdims=True)
    return x / np.maximum(norms, 1e-12)


def _hnsw_index(dim: int, M: int, metric: str):
    """
    Build IndexHNSWFlat with the requested metric.
    For FAISS versions without metric in constructor, fall back & try to set it.
    """
    metric = metric.lower()
    if metric not in {"ip", "l2"}:
        raise ValueError("metric must be 'ip' or 'l2'")
    METRIC = faiss.METRIC_INNER_PRODUCT if metric == "ip" else faiss.METRIC_L2

    # Try modern signature (d, M, metric)
    try:
        return faiss.IndexHNSWFlat(dim, M, METRIC)
    except TypeError:
        idx = faiss.IndexHNSWFlat(dim, M)
        # Some builds expose metric_type; if not, accept default (L2)
        try:
            idx.metric_type = METRIC  # may no-op on older builds
        except Exception:
            if metric == "ip":
                print("[WARN] Index metric could not be set to IP; "
                      "ensure vectors are L2-normalized and use IP-compatible search.")
        return idx


def _hash_bucket(val: str, n: int) -> int:
    """Stable shard id via SHA1(id) % n for better topic balance."""
    h = hashlib.sha1(val.encode("utf-8", errors="ignore")).hexdigest()
    return int(h[:8], 16) % n


def _value_counts_int(series: pd.Series) -> dict:
    """Return a plain dict of str->int counts (safe for JSON)."""
    vc = series.value_counts(dropna=False)
    # Convert NaN to "(null)" for readability
    keys = [("(null)" if (isinstance(k, float) and np.isnan(k)) else str(k)) for k in vc.index.tolist()]
    vals = [int(v) for v in vc.values.tolist()]
    return dict(zip(keys, vals))


def shard_index(
    n_shards: int = 8,
    metric: str = "ip",
    normalize: int = 1,
    M: int = 32,
    efC: int = 200,
    split: str = "hash",  # 'hash' or 'contiguous'
    seed: int = 42,
    save_vectors: int = 0,  # 1 = save vecs_shard_XXX.npy (optional)
):
    rng = np.random.default_rng(seed)

    meta_pq = f"{EMB_DIR}/multi_meta.parquet"
    vec_np  = f"{EMB_DIR}/multi_vectors.npy"

    if not os.path.exists(meta_pq) or not os.path.exists(vec_np):
        raise FileNotFoundError(
            f"Missing inputs:\n  {meta_pq if os.path.exists(meta_pq) else '(not found)'}\n"
            f"  {vec_np if os.path.exists(vec_np) else '(not found)'}"
        )

    print("Loading meta & vectors ...")
    meta = pd.read_parquet(meta_pq)
    vecs = np.load(vec_np).astype("float32")
    if len(meta) != len(vecs):
        raise ValueError(f"meta rows {len(meta)} != vectors {len(vecs)}")

    N, dim = vecs.shape
    print(f"Loaded {N:,} vectors (dim={dim})")

    # Decide domain column (for diagnostics)
    domain_col = "domain" if "domain" in meta.columns else ("source" if "source" in meta.columns else None)
    if domain_col is None:
        print("[INFO] No 'domain' or 'source' column found; domain counts will be omitted.")

    # Optional normalization (recommended for IP/cosine semantics)
    do_norm = bool(int(normalize))
    if do_norm:
        print("Normalizing vectors (L2) ...")
        vecs = _normalize_rows(vecs)

    # Compute shard assignment
    print(f"Sharding strategy: {split}  |  n_shards={n_shards}")
    if split == "hash":
        if "id" not in meta.columns:
            print("[WARN] meta has no 'id' column; using contiguous splits.")
            split = "contiguous"
        else:
            assign = meta["id"].astype(str).map(lambda s: _hash_bucket(s, n_shards)).to_numpy()
    if split == "contiguous":
        shard_size = int(np.ceil(N / n_shards))
        assign = np.repeat(np.arange(n_shards), shard_size)[:N]

    # Global domain counts (pre-sharding)
    global_domain_counts = None
    if domain_col is not None:
        global_domain_counts = _value_counts_int(meta[domain_col])

    # Manifest skeleton
    manifest = {
        "created_at": datetime.now(timezone.utc).isoformat(),
        "faiss_version": getattr(faiss, "__version__", "unknown"),
        "n_total": int(N),
        "n_shards": int(n_shards),
        "dim": int(dim),
        "normalized": bool(do_norm),
        "index_kind": "hnsw_flat",
        "index_params": {"M": int(M), "efConstruction": int(efC)},
        "index_metric": metric.lower(),  # logical metric used when building
        "split_strategy": split,
        "seed": int(seed),
        "save_vectors": bool(int(save_vectors)),
        "domain_counts_total": global_domain_counts if global_domain_counts is not None else {},
        "shards": [],
    }

    # Build each shard
    pbar = tqdm(range(n_shards), desc="Building shards")
    for i in pbar:
        mask = (assign == i)
        n_i = int(mask.sum())
        if n_i == 0:
            sub_meta = meta.iloc[0:0].copy()
            sub_vecs = np.zeros((0, dim), dtype="float32")
        else:
            sub_meta = meta.loc[mask].reset_index(drop=True)
            sub_vecs = vecs[mask].copy()

        shard_id = f"{i:03d}"
        meta_p = f"{OUT_EMB}/meta_shard_{shard_id}.parquet"
        idx_p  = f"{OUT_IDX}/shard_{shard_id}.faiss"
        vec_p  = f"{OUT_EMB}/vecs_shard_{shard_id}.npy"

        # Save meta
        sub_meta.to_parquet(meta_p)

        # Build FAISS HNSW
        index = _hnsw_index(dim, M, metric)
        try:
            index.hnsw.efConstruction = int(efC)
        except Exception:
            pass
        if n_i > 0:
            index.add(sub_vecs)

        # Persist index
        faiss.write_index(index, idx_p)

        # Optional: save vectors for this shard
        vectors_path = None
        if int(save_vectors) and n_i > 0:
            np.save(vec_p, sub_vecs)
            vectors_path = vec_p

        # Domain counts for this shard
        shard_domain_counts = {}
        if (domain_col is not None) and (n_i > 0):
            shard_domain_counts = _value_counts_int(sub_meta[domain_col])

        manifest["shards"].append({
            "id": shard_id,
            "n": n_i,
            "meta_path": meta_p,
            "index_path": idx_p,
            **({"vectors_path": vectors_path} if vectors_path else {}),
            "domain_counts": shard_domain_counts
        })
        pbar.set_postfix({"shard": shard_id, "rows": n_i})

    # Write manifest
    man_path = f"{OUT_IDX}/manifest.json"
    with open(man_path, "w", encoding="utf-8") as f:
        json.dump(manifest, f, indent=2)
    print("Manifest written:", man_path)
    print("Sum(ntotal) =", sum(s["n"] for s in manifest["shards"]))


if __name__ == "__main__":
    ap = argparse.ArgumentParser()
    ap.add_argument("--n_shards", type=int, default=8)
    ap.add_argument("--metric", type=str, default="ip", choices=["ip", "l2"])
    ap.add_argument("--normalize", type=int, default=1)   # 1=True (normalize rows)
    ap.add_argument("--M", type=int, default=32)
    ap.add_argument("--efC", type=int, default=200)
    ap.add_argument("--split", type=str, default="hash", choices=["hash", "contiguous"])
    ap.add_argument("--seed", type=int, default=42)
    ap.add_argument("--save_vectors", type=int, default=0)  # 1 to save vecs_shard_XXX.npy
    args = ap.parse_args()

    shard_index(
        n_shards=args.n_shards,
        metric=args.metric,
        normalize=args.normalize,
        M=args.M,
        efC=args.efC,
        split=args.split,
        seed=args.seed,
        save_vectors=args.save_vectors,
    )

Overwriting scripts/shard_index.py


Run the sharder (balanced, IP, normalized)

In [ ]:
!python scripts/shard_index.py --n_shards 8 --metric ip --normalize 1 --M 32 --efC 200 --split hash --seed 42 --save_vectors 1

Loading meta & vectors ...
Loaded 79,164 vectors (dim=384)
Normalizing vectors (L2) ...
Sharding strategy: hash  |  n_shards=8
Building shards: 100% 8/8 [00:42<00:00,  5.37s/it, shard=007, rows=9858]
Manifest written: /content/drive/MyDrive/AMS 560 PROJECT/indices/multi_default_sharded/manifest.json
Sum(ntotal) = 79164


Update configs/index/config.json using the manifest (with scaling knobs)

In [ ]:
import os, json

ROOT = "/content/drive/MyDrive/AMS 560 PROJECT"
CONFIG_PATH = os.path.join(ROOT, "configs/index/config.json")
MANIFEST_PATH = os.path.join(ROOT, "indices/multi_default_sharded/manifest.json")

# Helpers
def load_json(p):
    with open(p, "r", encoding="utf-8") as f:
        return json.load(f)

def save_json(p, obj):
    os.makedirs(os.path.dirname(p), exist_ok=True)
    with open(p, "w", encoding="utf-8") as f:
        json.dump(obj, f, indent=2)

def ensure_cfg_skeleton(cfg):
    if "indices" not in cfg or not isinstance(cfg["indices"], dict):
        cfg["indices"] = {}
    if "config_version" not in cfg:
        cfg["config_version"] = "1"
    return cfg

# Read manifest
man = load_json(MANIFEST_PATH)
idx_paths = [s["index_path"] for s in man["shards"]]
meta_paths = [s["meta_path"]  for s in man["shards"]]

assert len(idx_paths) == len(meta_paths) > 0, "manifest index/meta length mismatch"

# Load or create config
cfg = load_json(CONFIG_PATH) if os.path.exists(CONFIG_PATH) else {}
cfg = ensure_cfg_skeleton(cfg)

# Build/replace the multi_sharded block with knobs aligned to Step 3.2
cfg["indices"]["multi_sharded"] = {
    "backend": "faiss",
    "model_name": "BAAI/bge-small-en-v1.5",
    "normalize_query": True,               # queries normalized at search time
    "metric": "ip",                        # requested similarity metric (runtime)
    "index_metric": man.get("index_metric", "ip"),  # metric used at build time
    "vector_normed": bool(man.get("normalized", True)),
    "efSearch": 128,
    "index_paths": idx_paths,
    "meta_paths": meta_paths,
    "extras": {
        "alpha": 3,
        "dedup_by_uid": True,
        "faiss_num_threads": os.cpu_count() or 8,
        "text_fields": ["chunk_text", "text", "preview"]
    }
}

# Make sharded active
cfg["active_index"] = "multi_sharded"
cfg["config_version"] = "1"

save_json(CONFIG_PATH, cfg)

print("✓ Updated config at:", CONFIG_PATH)
print("\n--- multi_sharded summary ---")
print("active_index:", cfg.get("active_index"))
print("#index_paths:", len(cfg["indices"]["multi_sharded"]["index_paths"]))
print("#meta_paths :", len(cfg["indices"]["multi_sharded"]["meta_paths"]))
print("index_metric:", cfg["indices"]["multi_sharded"]["index_metric"], "| vector_normed:", cfg["indices"]["multi_sharded"]["vector_normed"])

✓ Updated config at: /content/drive/MyDrive/AMS 560 PROJECT/configs/index/config.json

--- multi_sharded summary ---
active_index: multi_sharded
#index_paths: 8
#meta_paths : 8
index_metric: ip | vector_normed: True


Validation

In [ ]:
from pipeline.runner import load_pipeline

CONFIG_PATH = "/content/drive/MyDrive/AMS 560 PROJECT/configs/index/config.json"

P = load_pipeline(CONFIG_PATH)  # uses active_index=multi_sharded
hits = P.search_one("Explain PageRank algorithm", top_k=5)
print(f"Got {len(hits)} hits (sharded). First title:", hits[0]["title"], "| source:", hits[0]["source"])
P.close()

[FAISS] Using 12 threads
Got 5 hits (sharded). First title: Search engine | source: wikipedia


Test Sharded Retrieval

In [ ]:
from pipeline.runner import load_pipeline
import json

P = load_pipeline("/content/drive/MyDrive/AMS 560 PROJECT/configs/index/config.json", index_key="multi_sharded")
hits = P.search_one("explain PageRank algorithm", top_k=5)
print(f"Got {len(hits)} hits (sharded). First:", hits[0])
P.close()

manifest = json.load(open("/content/drive/MyDrive/AMS 560 PROJECT/indices/multi_default_sharded/manifest.json"))
print("Sum(ntotal) =", sum(s["n"] for s in manifest["shards"]))

[FAISS] Using 12 threads
Got 5 hits (sharded). First: {'rank': 1, 'score': 0.7310386300086975, 'id': 'f52fac4b-452a-4449-8cb1-bae763c4f528', 'uid': 'f52fac4b-452a-4449-8cb1-bae763c4f528::c13', 'source': 'wikipedia', 'title': 'Search engine', 'url': 'https://en.wikipedia.org/wiki/Search%20engine', 'text': '', 'chunk_id': 13}
Sum(ntotal) = 79164


set project root so paths resolve

In [ ]:
import os, json, pathlib

# 👇 change this if your project lives somewhere else
PROJECT_ROOT = "/content/drive/MyDrive/AMS 560 PROJECT"
os.makedirs(f"{PROJECT_ROOT}/configs/index", exist_ok=True)

CFG_PATH      = f"{PROJECT_ROOT}/configs/index/config.json"
MANIFEST_PATH = f"{PROJECT_ROOT}/indices/multi_default_sharded/manifest.json"  # from Step 3.3/3.6
print("Config path:", CFG_PATH)
print("Manifest:", MANIFEST_PATH, "exists?", os.path.exists(MANIFEST_PATH))

Config path: /content/drive/MyDrive/AMS 560 PROJECT/configs/index/config.json
Manifest: /content/drive/MyDrive/AMS 560 PROJECT/indices/multi_default_sharded/manifest.json exists? True


helper: load/patch/save config JSON

In [ ]:
def load_json(p):
    with open(p, "r", encoding="utf-8") as f:
        return json.load(f)

def save_json(p, obj):
    with open(p, "w", encoding="utf-8") as f:
        json.dump(obj, f, indent=2)
    print("✅ wrote", p)

def ensure_cfg_skeleton(cfg):
    if "indices" not in cfg: cfg["indices"] = {}
    if "config_version" not in cfg: cfg["config_version"] = "1"
    return cfg

In [ ]:
# --- Read manifest (handles both formats)
man = load_json(MANIFEST_PATH)

if "index_paths" in man and "meta_paths" in man:
    # Flat list format (already there)
    index_paths = man["index_paths"]
    meta_paths  = man["meta_paths"]
else:
    # Shard list format → extract
    shards = man.get("shards", [])
    if not shards:
        raise ValueError("Manifest missing 'shards' array.")
    index_paths = [s["index_path"] for s in shards]
    meta_paths  = [s["meta_path"]  for s in shards]

assert len(index_paths) == len(meta_paths) > 0, "manifest index/meta length mismatch"

# --- Load or create config
cfg = load_json(CFG_PATH) if os.path.exists(CFG_PATH) else {}
cfg = ensure_cfg_skeleton(cfg)

# --- Fill the multi_sharded block with the new knobs
cfg["indices"]["multi_sharded"] = {
    "backend": "faiss",
    "model_name": "BAAI/bge-small-en-v1.5",
    "normalize_query": True,         # queries normalized at search time
    "metric": "ip",                  # requested similarity metric
    "index_metric": "ip",            # actual FAISS index semantics
    "vector_normed": True,           # your corpus vectors were L2-normalized at build time
    "efSearch": 128,
    "index_paths": index_paths,
    "meta_paths": meta_paths,
    "extras": {
        "alpha": 3,
        "dedup_by_uid": True,
        "faiss_num_threads": os.cpu_count() or 8,
        "text_fields": ["chunk_text", "text", "preview"]
    }
}

cfg["active_index"]   = "multi_sharded"
cfg["config_version"] = "1"

save_json(CFG_PATH, cfg)

print("✓ Patched config:", CFG_PATH)
print("  shards:", len(index_paths))
print("  sample index:", index_paths[0])
print("  sample meta :", meta_paths[0])

✅ wrote /content/drive/MyDrive/AMS 560 PROJECT/configs/index/config.json
✓ Patched config: /content/drive/MyDrive/AMS 560 PROJECT/configs/index/config.json
  shards: 8
  sample index: /content/drive/MyDrive/AMS 560 PROJECT/indices/multi_default_sharded/shard_000.faiss
  sample meta : /content/drive/MyDrive/AMS 560 PROJECT/data/embeddings/multi_default_sharded/meta_shard_000.parquet


patch the sharded index block (indices.multi_sharded)

In [ ]:
FAISS_SINGLE = f"{PROJECT_ROOT}/indices/multi_default/multi_hnsw.faiss"
META_SINGLE  = f"{PROJECT_ROOT}/data/embeddings/multi_default/multi_meta.parquet"

cfg = load_json(CFG_PATH)
cfg = ensure_cfg_skeleton(cfg)

cfg["indices"]["multi_default"] = {
    "backend": "faiss",
    "model_name": "BAAI/bge-small-en-v1.5",
    "normalize_query": True,
    "metric": "ip",
    "index_metric": "ip",      # leave 'ip' since corpus is normalized
    "vector_normed": True,     # ✅ your corpus was normalized
    "efSearch": 128,
    "faiss_path": FAISS_SINGLE,
    "meta_path": META_SINGLE,
    "extras": {
        "faiss_num_threads": os.cpu_count() or 8,
        "text_fields": ["chunk_text", "text", "preview"]
    }
}

# (Optional) switch active_index:
# cfg["active_index"] = "multi_default"

save_json(CFG_PATH, cfg)

✅ wrote /content/drive/MyDrive/AMS 560 PROJECT/configs/index/config.json


verify config loads cleanly with your loader

In [ ]:
# Uses your Step 3.1 loader (already in your repo)
from pipeline.config import load_config
raw, nic = load_config(CFG_PATH)  # uses active_index
print("Active index:", raw.get("active_index"))
print("Index metric:", nic.index_metric, "| vector_normed:", nic.vector_normed, "| normalize_query:", nic.normalize_query)
print("Sharded?" , bool(nic.index_paths))
if nic.index_paths:
    print("#shards:", len(nic.index_paths))

Active index: multi_sharded
Index metric: ip | vector_normed: True | normalize_query: True
Sharded? True
#shards: 8


## Step 3.4 — Chroma Backend Integration

**Objective**

Create a Chroma collection (multi_default) using merged embeddings and metadata,
and add a backend implementation compatible with existing pipeline abstraction.

Setup

In [ ]:
!mkdir -p scripts

In [ ]:
import os
os.makedirs("configs/index", exist_ok=True)
os.makedirs("pipeline/backends", exist_ok=True)
os.makedirs("scripts", exist_ok=True)
os.makedirs("indices/chroma_store", exist_ok=True)  # persistent Chroma dir on Drive

Config loader that permits non-FAISS backends

In [ ]:
%%writefile pipeline/config.py
"""
Config loader for retrieval pipeline (scaling-ready).

- Supports JSON (.json) and YAML (.yml/.yaml)
- Validates index blocks (single-index or sharded) for FAISS backends
- Normalizes keys so backends see a consistent shape
- Adds schema versioning and metric/normalization semantics
- Supports remote paths (s3://, gs://, hdfs://) and optional path checks
"""

from __future__ import annotations
import json
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Tuple
import os

try:
    import yaml  # type: ignore
except Exception as e:
    raise ImportError(
        "pyyaml is required. Install with: pip install pyyaml"
    ) from e


class ConfigError(Exception):
    """Raised when configuration is missing/invalid."""


# Accepted config schema versions (simple gate for now)
_ACCEPTED_CONFIG_VERSIONS = {"1"}


@dataclass
class NormalizedIndexConfig:
    # core knobs
    backend: str
    model_name: str
    normalize_query: bool
    metric: str
    efSearch: int

    # union of single vs sharded (for FAISS backends)
    faiss_path: Optional[str] = None
    meta_path: Optional[str] = None
    index_paths: Optional[List[str]] = None
    meta_paths: Optional[List[str]] = None

    # semantics (for cross-backend parity & validation)
    index_metric: Optional[str] = None         # 'ip' | 'l2' | 'cosine'
    vector_normed: Optional[bool] = None       # True if stored vectors are unit-normalized

    # free-form passthrough (if backends need extras)
    extras: Dict[str, Any] = field(default_factory=dict)

    # loader context (useful for validation/telemetry)
    index_key: Optional[str] = None
    config_version: Optional[str] = None
    skip_path_checks: bool = False


def _load_raw_config(path: str) -> Dict[str, Any]:
    if not os.path.exists(path):
        raise ConfigError(f"Config file not found: {path}")
    _, ext = os.path.splitext(path.lower())
    try:
        with open(path, "r", encoding="utf-8") as f:
            if ext == ".json":
                return json.load(f)
            if ext in (".yml", ".yaml"):
                return yaml.safe_load(f)
            # try JSON first then YAML if unknown extension
            try:
                f.seek(0)
                return json.load(f)
            except Exception:
                f.seek(0)
                return yaml.safe_load(f)
    except Exception as e:
        raise ConfigError(f"Failed to parse config {path}: {e}") from e


def _require(d: Dict[str, Any], key: str, ctx: str) -> Any:
    if key not in d:
        raise ConfigError(f"Missing required key '{key}' in {ctx}")
    return d[key]


def _as_bool(v: Any, default: bool) -> bool:
    if v is None:
        return default
    if isinstance(v, bool):
        return v
    if isinstance(v, (int, float)):
        return bool(v)
    if isinstance(v, str):
        s = v.strip().lower()
        if s in {"1", "true", "yes", "y", "on"}:
            return True
        if s in {"0", "false", "no", "n", "off"}:
            return False
    raise ConfigError(f"Expected boolean-like value, got: {v!r}")


def _as_int(v: Any, default: int) -> int:
    if v is None:
        return default
    if isinstance(v, int):
        return v
    if isinstance(v, float):
        return int(v)
    if isinstance(v, str):
        s = v.strip()
        try:
            # tolerate "128 ", "1e2", etc.
            return int(float(s))
        except Exception:
            pass
    raise ConfigError(f"Expected integer-like value, got: {v!r}")


def _is_remote_path(p: str) -> bool:
    p = (p or "").lower()
    return p.startswith("s3://") or p.startswith("gs://") or p.startswith("hdfs://")


def _validate_paths_exist(paths: List[str], ctx: str, *, skip_checks: bool) -> None:
    if skip_checks:
        return
    missing = [
        p for p in paths
        if (not _is_remote_path(p)) and (not os.path.exists(p))
    ]
    if missing:
        raise ConfigError(f"{ctx}: missing files:\n  - " + "\n  - ".join(missing))


def _normalize_index_block(
    block: Dict[str, Any],
    ctx: str,
    index_key: str,
    top_level_skip_checks: bool,
) -> NormalizedIndexConfig:
    # knobs
    backend = (block.get("backend") or "faiss").lower()
    model_name = block.get("model_name", "BAAI/bge-small-en-v1.5")
    metric = (block.get("metric") or "ip").lower()
    efSearch = _as_int(block.get("efSearch"), 64)
    normalize_query = _as_bool(block.get("normalize_query"), True)

    # semantics (extra flags that help enforce parity across backends)
    index_metric = (block.get("index_metric") or metric or "").lower() or None
    vector_normed = block.get("vector_normed")
    if vector_normed is not None:
        vector_normed = _as_bool(vector_normed, False)

    # support single-index OR sharded-index (for FAISS backends)
    faiss_path = block.get("faiss_path")
    meta_path = block.get("meta_path")
    index_paths = block.get("index_paths")
    meta_paths = block.get("meta_paths")

    # per-block override for skipping path checks (e.g. cluster/remote mode)
    skip_path_checks = _as_bool(block.get("skip_path_checks"), top_level_skip_checks)

    # -------------------------------------------------------------
    # Only enforce single/sharded + path checks for FAISS backends.
    # For other backends (e.g., "chroma"), faiss_path/index_paths
    # are optional and can be omitted entirely.
    # -------------------------------------------------------------
    if backend == "faiss":
        single_ok = bool(faiss_path and meta_path)
        shard_ok = bool(index_paths and meta_paths)

        if single_ok and shard_ok:
            raise ConfigError(
                f"{ctx}: Provide EITHER single (faiss_path/meta_path) OR "
                f"sharded (index_paths/meta_paths), not both."
            )
        if not (single_ok or shard_ok):
            raise ConfigError(
                f"{ctx}: Must provide single (faiss_path+meta_path) OR "
                f"sharded (index_paths+meta_paths)."
            )

        # path existence checks
        if single_ok:
            _validate_paths_exist(
                [faiss_path, meta_path],
                f"{ctx} (single)",
                skip_checks=skip_path_checks,
            )
        else:
            if not isinstance(index_paths, list) or not isinstance(meta_paths, list):
                raise ConfigError(
                    f"{ctx}: index_paths/meta_paths must be lists for sharded mode."
                )
            if len(index_paths) != len(meta_paths):
                raise ConfigError(
                    f"{ctx}: index_paths and meta_paths length mismatch "
                    f"({len(index_paths)} vs {len(meta_paths)})."
                )
            _validate_paths_exist(
                index_paths,
                f"{ctx} index_paths",
                skip_checks=skip_path_checks,
            )
            _validate_paths_exist(
                meta_paths,
                f"{ctx} meta_paths",
                skip_checks=skip_path_checks,
            )

    # everything not core becomes extras
    extras = {
        k: v
        for k, v in block.items()
        if k
        not in {
            "backend",
            "model_name",
            "normalize_query",
            "metric",
            "efSearch",
            "faiss_path",
            "meta_path",
            "index_paths",
            "meta_paths",
            "index_metric",
            "vector_normed",
            "skip_path_checks",
        }
    }

    return NormalizedIndexConfig(
        backend=backend,
        model_name=model_name,
        normalize_query=normalize_query,
        metric=metric,
        efSearch=efSearch,
        faiss_path=faiss_path,
        meta_path=meta_path,
        index_paths=index_paths,
        meta_paths=meta_paths,
        index_metric=index_metric,
        vector_normed=vector_normed,
        extras=extras,
        index_key=index_key,
        config_version=None,         # filled by load_config
        skip_path_checks=skip_path_checks,
    )


def load_config(path: str, index_key: Optional[str] = None) -> Tuple[Dict[str, Any], NormalizedIndexConfig]:
    """
    Returns (raw_config_dict, normalized_index_cfg) for the chosen index key.
    - If index_key is None, uses raw['active_index'].
    - Validates presence and correctness of the selected index block.
    - Enforces schema version and provides remote-path/skip-check behavior.
    """
    raw = _load_raw_config(path)
    if not isinstance(raw, dict):
        raise ConfigError("Top-level config must be an object/dict.")

    # Schema versioning
    cfg_version = str(raw.get("config_version", "1")).strip()
    if cfg_version not in _ACCEPTED_CONFIG_VERSIONS:
        raise ConfigError(
            f"Unsupported config_version='{cfg_version}'. "
            f"Accepted versions: {_ACCEPTED_CONFIG_VERSIONS}"
        )

    indices = _require(raw, "indices", "root")
    if not isinstance(indices, dict) or not indices:
        raise ConfigError("'indices' must be a non-empty object.")

    if index_key is None:
        index_key = raw.get("active_index")
        if not index_key:
            raise ConfigError("Provide index_key or set 'active_index' in config.")

    if index_key not in indices:
        raise ConfigError(f"Index '{index_key}' not found in 'indices'.")

    block = indices[index_key]
    if not isinstance(block, dict):
        raise ConfigError(f"Index '{index_key}' must be an object.")

    # allow a top-level default for skipping path checks (e.g., cluster mode)
    top_level_skip_checks = _as_bool(raw.get("skip_path_checks"), False)

    nic = _normalize_index_block(
        block,
        f"index '{index_key}'",
        index_key,
        top_level_skip_checks,
    )
    # fill loader context fields
    nic.config_version = cfg_version

    if nic.index_metric and nic.metric and nic.index_metric != nic.metric:
        import warnings
        warnings.warn(
            f"Requested metric '{nic.metric}' differs from index_metric '{nic.index_metric}'. "
            "Ensure query normalization / FAISS setup matches."
        )

    return raw, nic


Overwriting pipeline/config.py


Backend interface

In [ ]:
%%writefile pipeline/backends/base.py
from __future__ import annotations
from typing import Any, Dict, List, Protocol, runtime_checkable
Result = Dict[str, Any]

@runtime_checkable
class VectorBackend(Protocol):
    def load(self, cfg: Any) -> None: ...
    def encode(self, text_or_list: Any) -> Any: ...
    def search(self, qvec: Any, top_k: int = 5, filter_source: str | None = None) -> List[Result]: ...
    def close(self) -> None: ...

Overwriting pipeline/backends/base.py


Chroma ingestion (build persistent collection)

In [ ]:
%%writefile scripts/chroma_ingest.py
import os, argparse, json
import numpy as np, pandas as pd
from tqdm import tqdm
import chromadb
from chromadb.config import Settings

MODEL_NAME = "BAAI/bge-small-en-v1.5"
VECTORS_ARE_NORMALIZED = True  # ✅ you already normalized corpus vectors at build time

def _build_unique_ids(meta: pd.DataFrame) -> pd.Series:
    if "chunk_id" in meta.columns:
        ch = meta["chunk_id"].fillna(-1).astype("int64").astype(str)
        uids = meta["id"].astype(str) + "::c" + ch
    else:
        uids = meta["id"].astype(str) + "::row" + pd.Series(range(len(meta)), index=meta.index).astype(str)
    if not uids.is_unique:
        uids = uids + "::" + pd.Series(range(len(uids)), index=uids.index).astype(str)
    return uids

def _sanitize_metadata(meta: pd.DataFrame) -> list[dict]:
    m = meta.copy()

    # Normalize domain/source consistently (lowercase, strip)
    if "domain" in m.columns:
        m["domain"] = m["domain"].astype(str).str.strip().str.lower()
    if "source" in m.columns:
        m["source"] = m["source"].astype(str).str.strip().str.lower()

    # Ensure common string fields are strings
    for col in ["title", "url", "source_id", "id"]:
        if col in m.columns:
            m[col] = m[col].astype(str)

    # chunk_id as int or None
    if "chunk_id" in m.columns:
        m["chunk_id"] = m["chunk_id"].apply(lambda x: int(x) if pd.notna(x) else None)

    return m.to_dict(orient="records")

def ingest_chroma(project_root: str, collection_name: str, persist_dir: str):
    emb_dir = f"{project_root}/data/embeddings/multi_default"
    meta_pq = f"{emb_dir}/multi_meta.parquet"
    vec_np  = f"{emb_dir}/multi_vectors.npy"
    if not (os.path.exists(meta_pq) and os.path.exists(vec_np)):
        raise FileNotFoundError(f"Missing merged files:\n- {meta_pq}\n- {vec_np}")

    meta = pd.read_parquet(meta_pq)
    vecs = np.load(vec_np).astype("float32")
    if len(meta) != len(vecs):
        raise ValueError(f"meta/vector length mismatch: {len(meta)} vs {len(vecs)}")

    # Normalize domain/source columns BEFORE batching (helps server-side filtering)
    if "domain" in meta.columns:
        meta["domain"] = meta["domain"].astype(str).str.strip().str.lower()
    elif "source" in meta.columns:
        meta["source"] = meta["source"].astype(str).str.strip().str.lower()

    os.environ["CHROMA_TELEMETRY_ENABLED"] = "false"
    os.makedirs(persist_dir, exist_ok=True)
    client = chromadb.PersistentClient(path=persist_dir, settings=Settings(anonymized_telemetry=False))

    # Clean create the collection
    try:
        if collection_name in [c.name for c in client.list_collections()]:
            client.delete_collection(collection_name)
    except Exception:
        pass
    coll = client.create_collection(name=collection_name)

    ids = _build_unique_ids(meta)
    print(f"Ingesting {len(meta):,} into '{collection_name}' @ {persist_dir}")
    bs = 1000
    for i in tqdm(range(0, len(meta), bs)):
        batch = meta.iloc[i:i+bs]
        coll.add(
            ids=ids.iloc[i:i+bs].tolist(),
            embeddings=vecs[i:i+bs].tolist(),
            metadatas=_sanitize_metadata(batch)
        )

    info = {
        "collection": collection_name,
        "persist_dir": persist_dir,
        "n_records": len(meta),
        "fields": list(meta.columns),
        "id_format": "uid = id::c<chunk_id> (or ::row<i>)",
        "model_name": MODEL_NAME,
        "vector_normed": VECTORS_ARE_NORMALIZED,
        "metric": "cosine",  # Chroma uses cosine distance
        "score_semantics": "distance=1-cosine_similarity; report similarity=1-distance"
    }
    man_path = f"{project_root}/indices/{collection_name}_chroma_manifest.json"
    with open(man_path, "w", encoding="utf-8") as f:
        json.dump(info, f, indent=2)
    print("✓ Manifest:", man_path)
    print("✓ Done.")

if __name__ == "__main__":
    ap = argparse.ArgumentParser()
    ap.add_argument("--collection", default="multi_default")
    ap.add_argument("--persist_dir", required=True)
    args = ap.parse_args()
    ingest_chroma("/content/drive/MyDrive/AMS 560 PROJECT", args.collection, args.persist_dir)

Overwriting scripts/chroma_ingest.py


Run ingestion

In [ ]:
!python scripts/chroma_ingest.py --collection multi_default --persist_dir "indices/chroma_store"

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given
Ingesting 79,164 into 'multi_default' @ indices/chroma_store
  0% 0/80 [00:00<?, ?it/s]Failed to send telemetry event CollectionAddEvent: capture() takes 1 positional argument but 3 were given
100% 80/80 [04:10<00:00,  3.13s/it]
✓ Manifest: /content/drive/MyDrive/AMS 560 PROJECT/indices/multi_default_chroma_manifest.json
✓ Done.


Chroma backend

In [ ]:
%%writefile pipeline/backends/chroma_backend.py
"""
Chroma backend implementation matching VectorBackend interface.
- Converts Chroma cosine distance → similarity (score = 1 - distance)
- Validates collection/model/normalization parity via manifest
- Adaptive overfetch when filtering; de-dup by uid
- Stable uid mapping: returns both 'uid' and base 'id'
"""

from __future__ import annotations
import os, json
from typing import Any, List, Dict, Optional

import chromadb
from chromadb.config import Settings
import numpy as np
from sentence_transformers import SentenceTransformer

from pipeline.backends.base import VectorBackend, Result


class ChromaBackend(VectorBackend):
    def __init__(self):
        self.client = None
        self.collection = None
        self.model: Optional[SentenceTransformer] = None
        self.cfg = None
        self.text_fields_pref: List[str] = ["chunk_text", "text", "preview"]
        self.alpha: int = 3
        self.dedup_by_uid: bool = True
        self.collection_name: str = "multi_default"
        self.persist_dir: Optional[str] = None
        self.manifest: Dict[str, Any] = {}

    def _resolve_extras(self, cfg) -> dict:
        ex = getattr(cfg, "extras", {}) or {}
        if isinstance(ex, dict) and "persist_dir" not in ex and "extras" in ex and isinstance(ex["extras"], dict):
            ex = ex["extras"]
        return ex

    def _load_manifest_if_exists(self):
        # Try to read indices/<collection>_chroma_manifest.json for parity checks
        if not self.persist_dir:
            return
        man_path = os.path.join(os.path.dirname(self.persist_dir.rstrip("/")), f"{self.collection_name}_chroma_manifest.json")
        # If persist_dir is ".../indices/chroma_store", manifest path above points to ".../indices/multi_default_chroma_manifest.json"
        alt_path = os.path.join(self.persist_dir, f"{self.collection_name}_chroma_manifest.json")
        for p in [man_path, alt_path]:
            if os.path.exists(p):
                try:
                    with open(p, "r", encoding="utf-8") as f:
                        self.manifest = json.load(f)
                except Exception:
                    self.manifest = {}
                break

    def _assert_parity(self):
        # Only assert if we found a manifest
        if not self.manifest:
            return
        want_model = getattr(self.cfg, "model_name", None)
        have_model = self.manifest.get("model_name")
        if want_model and have_model and want_model != have_model:
            raise RuntimeError(
                f"Model mismatch for Chroma collection '{self.collection_name}': "
                f"config.model_name='{want_model}' vs manifest.model_name='{have_model}'. "
                f"Re-ingest with scripts/chroma_ingest.py or switch config."
            )
        # Vector normalization parity
        vec_normed = self.manifest.get("vector_normed")
        if vec_normed is not None and not bool(vec_normed):
            # You said your corpus is normalized; if manifest claims otherwise, warn/raise.
            raise RuntimeError(
                f"Manifest indicates vector_normed={vec_normed} but config expects normalized corpus vectors. "
                f"Rebuild or verify ingestion."
            )
        # Count sanity (best-effort)
        try:
            coll_count = self.collection.count()
            man_count = int(self.manifest.get("n_records", coll_count))
            if abs(coll_count - man_count) > 0:
                raise RuntimeError(
                    f"Collection count ({coll_count}) != manifest n_records ({man_count}). "
                    f"Re-ingest to synchronize."
                )
        except Exception:
            pass

    def load(self, cfg):
        self.cfg = cfg
        ex = self._resolve_extras(cfg)

        self.persist_dir = ex.get("persist_dir")
        if not self.persist_dir:
            raise RuntimeError("Chroma backend requires extras.persist_dir in config.")

        self.collection_name = ex.get("collection", "multi_default")
        self.alpha = int(ex.get("alpha", 3))
        self.dedup_by_uid = bool(ex.get("dedup_by_uid", True))
        # optional text field preference
        if isinstance(ex.get("text_fields"), list) and ex["text_fields"]:
            self.text_fields_pref = list(ex["text_fields"])

        # Persistent client
        os.environ["CHROMA_TELEMETRY_ENABLED"] = "false"
        self.client = chromadb.PersistentClient(path=self.persist_dir, settings=Settings(anonymized_telemetry=False))

        try:
            self.collection = self.client.get_collection(self.collection_name)
        except Exception as e:
            raise RuntimeError(
                f"Chroma collection '{self.collection_name}' not found in '{self.persist_dir}'. "
                f"Run: python scripts/chroma_ingest.py --collection {self.collection_name} --persist_dir '{self.persist_dir}'"
            ) from e

        # Model
        self.model = SentenceTransformer(cfg.model_name)

        # Parity checks
        self._load_manifest_if_exists()
        self._assert_parity()

        print(f"Loaded Chroma collection: {self.collection_name} | count={self.collection.count()} | store={self.persist_dir}")

    def encode(self, text_or_list):
        if isinstance(text_or_list, str):
            texts = [text_or_list]
        else:
            texts = list(text_or_list)
        vecs = self.model.encode(
            texts,
            normalize_embeddings=getattr(self.cfg, "normalize_query", True)
        ).astype("float32")
        return vecs

    def _pick_text(self, meta: Dict[str, Any]) -> str:
        for k in self.text_fields_pref:
            if k in meta and meta[k]:
                return meta[k]
        return ""

    def _split_uid(self, uid: str) -> (str, Optional[int]):
        # uid format: "<id>::c<chunk_id>" or "<id>::row<i>"
        if "::c" in uid:
            base, suf = uid.split("::c", 1)
            try:
                return base, int(suf)
            except Exception:
                return base, None
        if "::row" in uid:
            base, _ = uid.split("::row", 1)
            return base, None
        return uid, None

    def search(self, qvec, top_k: int = 5, filter_source: str | None = None) -> List[Result]:
    # Chroma returns cosine distances; convert to similarity for consistency
      overfetch = top_k * (self.alpha if filter_source else 1)
      out: List[Result] = []
      seen_uid = set()

      # --- Normalize filter_source for case-insensitive matching ---
      fs = filter_source.strip().lower() if filter_source else None
      where = {"domain": fs} if fs else None

      query = qvec[0].tolist()
      results = self.collection.query(
          query_embeddings=[query],
          n_results=overfetch,
          where=where
      )

      ids = results.get("ids", [[]])[0]
      dists = results.get("distances", [[]])[0]  # cosine distance
      metas = results.get("metadatas", [[]])[0]

      for uid, dist, md in zip(ids, dists, metas):
          if uid is None:
              continue
          sim = 1.0 - float(dist)  # similarity = 1 - distance
          meta = md or {}

          # Extract source robustly and compare case-insensitively
          source = (meta.get("domain") or meta.get("source") or "").strip().lower()
          if fs and source != fs:
              continue

          if self.dedup_by_uid:
              if uid in seen_uid:
                  continue
              seen_uid.add(uid)

          base_id, chunk_id = self._split_uid(uid)

          out.append({
              "rank": len(out) + 1,
              "score": sim,                          # report similarity (higher is better)
              "id": base_id,                         # base document id
              "uid": uid,                            # stable chunk-level id
              "source": source,                      # already normalized lower-case
              "title": meta.get("title", ""),
              "url": meta.get("url", ""),
              "text": self._pick_text(meta),
              "chunk_id": chunk_id
          })
          if len(out) >= top_k:
              break
      return out


    def close(self):
        try:
            if self.client and hasattr(self.client, "reset"):
                self.client.reset()
        except Exception:
            pass
        self.client = None
        self.collection = None
        self.model = None


Overwriting pipeline/backends/chroma_backend.py


Minimal runner (register Chroma)

In [ ]:
%%writefile pipeline/runner.py
from __future__ import annotations
from typing import Optional, List, Dict, Any

from pipeline.config import load_config, NormalizedIndexConfig
from pipeline.backends.base import VectorBackend
from pipeline.backends.chroma_backend import ChromaBackend

_REGISTRY = {
    "chroma": ChromaBackend,
    # "faiss": FaissBackend,  # plug later if needed
}

class Pipeline:
    def __init__(self, backend: VectorBackend, nicfg: NormalizedIndexConfig):
        self.backend = backend
        self.nicfg = nicfg

    def search_one(self, query: str, top_k: int = 5, filter_source: str | None = None) -> List[Dict[str, Any]]:
        qvec = self.backend.encode(query)
        return self.backend.search(qvec, top_k=top_k, filter_source=filter_source)

    def search_many(self, queries: List[str], top_k: int = 5, filter_source: str | None = None) -> Dict[str, List[Dict[str, Any]]]:
        qvecs = self.backend.encode(queries)
        out = {}
        for i, q in enumerate(queries):
            out[q] = self.backend.search(qvecs[i:i+1], top_k=top_k, filter_source=filter_source)
        return out

    def close(self):
        self.backend.close()

def load_pipeline(cfg_path: str, index_key: Optional[str] = None, backend_override: Optional[str] = None) -> Pipeline:
    raw, nic = load_config(cfg_path, index_key=index_key)
    bkey = (backend_override or nic.backend or "chroma").lower()
    if bkey not in _REGISTRY:
        raise ValueError(f"Unknown backend '{bkey}'. Available: {list(_REGISTRY)}")
    backend = _REGISTRY[bkey]()  # type: ignore
    backend.load(nic)
    return Pipeline(backend, nic)

Overwriting pipeline/runner.py


Config JSON for Chroma (make it active)

In [ ]:
import json, os

cfg_path = "configs/index/config.json"
with open(cfg_path) as f:
    cfg = json.load(f)

# Ensure indices exists
cfg.setdefault("indices", {})

# Ensure a multi_chroma block exists and has the required fields
mc = cfg["indices"].get("multi_chroma", {})
mc.setdefault("backend", "chroma")
mc.setdefault("model_name", "BAAI/bge-small-en-v1.5")
mc.setdefault("normalize_query", True)
mc.setdefault("metric", "ip")
mc.setdefault("efSearch", 64)

# >>>>> THIS IS THE IMPORTANT PART <<<<<
extras = mc.get("extras", {})
extras.setdefault("collection", "multi_default")
extras["persist_dir"] = "indices/chroma_store"   # REQUIRED
extras.setdefault("alpha", 3)                    # optional overfetch for filtered queries
extras.setdefault("dedup_by_uid", True)          # optional de-dup behavior
extras.setdefault("text_fields", ["chunk_text","text","preview"])  # optional field preference
mc["extras"] = extras

cfg["indices"]["multi_chroma"] = mc
cfg["active_index"] = "multi_chroma"

with open(cfg_path, "w") as f:
    json.dump(cfg, f, indent=2)

print("Patched config at:", cfg_path)
print(json.dumps(cfg["indices"]["multi_chroma"], indent=2))

Patched config at: configs/index/config.json
{
  "backend": "chroma",
  "model_name": "BAAI/bge-small-en-v1.5",
  "normalize_query": true,
  "metric": "ip",
  "efSearch": 64,
  "extras": {
    "collection": "multi_default",
    "persist_dir": "indices/chroma_store",
    "alpha": 3,
    "dedup_by_uid": true,
    "text_fields": [
      "chunk_text",
      "text",
      "preview"
    ]
  }
}


Reload & sanity test (Chroma)

In [ ]:
!rm -rf /content/pipeline

In [ ]:
%cd "/content/drive/MyDrive/AMS 560 PROJECT"

/content/drive/MyDrive/AMS 560 PROJECT


In [ ]:
import pipeline.config
print(pipeline.config.__file__)

/content/drive/MyDrive/AMS 560 PROJECT/pipeline/config.py


In [ ]:
import sys
for m in list(sys.modules):
    if m.startswith("pipeline."):
        del sys.modules[m]

from pipeline.runner import load_pipeline

P = load_pipeline("configs/index/config.json")  # active_index=multi_chroma
hits = P.search_one("Explain PageRank algorithm", top_k=5)
print(f"Got {len(hits)} hits (Chroma). First:", hits[0]["title"], "| source:", hits[0]["source"])

hits_pubmed = P.search_one("metformin mechanism of action", top_k=5, filter_source="pubmed")
print("pubmed first:", hits_pubmed[0]["source"], "|", hits_pubmed[0]["title"])

P.close()

ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given


Loaded Chroma collection: multi_default | count=79164 | store=indices/chroma_store


ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given


Got 5 hits (Chroma). First: Search engine | source: wikipedia
pubmed first: pubmed | The beneficial effects of empagliflozin-metformin combination on cardiovascular risk factors in type 2 diabetes mellitus patients.


## **Step 3.5 — Benchmark harness + logging**

Ensure both backends are registered in the runner

In [ ]:
%%writefile pipeline/runner.py
from __future__ import annotations
from typing import Optional, List, Dict, Any

from pipeline.config import load_config, NormalizedIndexConfig
from pipeline.backends.base import VectorBackend
from pipeline.backends.chroma_backend import ChromaBackend
from pipeline.backends.faiss_backend import FaissBackend  # ensure this file exists

_REGISTRY = {
    "chroma": ChromaBackend,
    "faiss": FaissBackend,
}

class Pipeline:
    def __init__(self, backend: VectorBackend, nicfg: NormalizedIndexConfig):
        self.backend = backend
        self.nicfg = nicfg

    def search_one(self, query: str, top_k: int = 5, filter_source: str | None = None) -> List[Dict[str, Any]]:
        qvec = self.backend.encode(query)
        return self.backend.search(qvec, top_k=top_k, filter_source=filter_source)

    def search_many(self, queries: List[str], top_k: int = 5, filter_source: str | None = None) -> Dict[str, List[Dict[str, Any]]]:
        qvecs = self.backend.encode(queries)
        out = {}
        for i, q in enumerate(queries):
            out[q] = self.backend.search(qvecs[i:i+1], top_k=top_k, filter_source=filter_source)
        return out

    def close(self):
        self.backend.close()

def load_pipeline(cfg_path: str, index_key: Optional[str] = None, backend_override: Optional[str] = None) -> Pipeline:
    raw, nic = load_config(cfg_path, index_key=index_key)
    bkey = (backend_override or nic.backend or "faiss").lower()
    if bkey not in _REGISTRY:
        raise ValueError(f"Unknown backend '{bkey}'. Available: {list(_REGISTRY)}")
    backend = _REGISTRY[bkey]()  # type: ignore
    backend.load(nic)
    return Pipeline(backend, nic)

Overwriting pipeline/runner.py


bench.py (timestamped outputs, env capture, score semantics, optional quality)

In [ ]:
rm -rf results/multi_sharded

In [ ]:
%%writefile bench.py

import argparse, os, sys, json, time, csv, math, datetime, hashlib, platform
from typing import List, Dict, Any, Optional, Tuple
from pathlib import Path

import numpy as np

# Optional libs for env capture
try:
    import faiss
    _FAISS_OK = True
except Exception:
    _FAISS_OK = False

try:
    import chromadb
    _CHROMA_VER = getattr(chromadb, "__version__", "unknown")
except Exception:
    _CHROMA_VER = "unknown"

try:
    from sentence_transformers import __version__ as _ST_VER
except Exception:
    _ST_VER = "unknown"

from pipeline.runner import load_pipeline
from pipeline.config import load_config, ConfigError

# ---------------- util ---------------- #

def read_queries(qfile: str) -> List[str]:
    qs = []
    with open(qfile, "r", encoding="utf-8") as f:
        for ln in f:
            ln = ln.strip()
            if ln:
                qs.append(ln)
    if not qs:
        raise SystemExit(f"No queries found in {qfile}")
    return qs

def sha1_of_json(obj: Any) -> str:
    s = json.dumps(obj, sort_keys=True, ensure_ascii=False)
    return hashlib.sha1(s.encode("utf-8")).hexdigest()[:12]

def percentile(values: List[float], p: float) -> float:
    if not values:
        return 0.0
    values_sorted = sorted(values)
    k = (len(values_sorted)-1) * (p/100.0)
    f = math.floor(k); c = math.ceil(k)
    if f == c: return values_sorted[int(k)]
    d0 = values_sorted[f] * (c-k); d1 = values_sorted[c] * (k-f)
    return d0 + d1

def flat_hits_rows(query, hits, iter_id):
    rows = []
    for h in hits:
        rows.append({
            "query": query,
            "iter": iter_id,
            "rank": h.get("rank"),
            "score": h.get("score"),
            "id": h.get("id"),
            "uid": h.get("uid"),
            "source": h.get("source"),
            "title": h.get("title"),
            "url": h.get("url"),
            "chunk_id": h.get("chunk_id"),
            "text": h.get("text"),
        })
    return rows

def load_truth_map(path: Optional[str]) -> Dict[str, List[str]]:
    """
    Optional gold file (JSON) mapping query -> list of relevant uids/ids.
    Example:
      {"Explain PageRank algorithm": ["doc123::c0","doc456::c2", ...], ...}
    """
    if not path: return {}
    with open(path, "r", encoding="utf-8") as f:
        return json.load(f)

def compute_recall_mrr(hits_by_query: Dict[str, List[Dict[str, Any]]], gold: Dict[str, List[str]]) -> Tuple[Optional[float], Optional[float]]:
    """
    Computes recall@k and MRR using uid if present, else falls back to id.
    """
    recall_list = []
    rr_list = []
    for q, hits in hits_by_query.items():
        if q not in gold:
            continue
        gold_set = set(gold[q])
        if not gold_set:
            continue
        seq = [h.get("uid") or h.get("id") for h in hits]
        seq = [x for x in seq if x]
        hit_any = any(s in gold_set for s in seq)
        if hit_any:
            rank = None
            for i, s in enumerate(seq, start=1):
                if s in gold_set:
                    rank = i
                    break
            rr_list.append(1.0 / rank if rank else 0.0)
        else:
            rr_list.append(0.0)
        retrieved = set(seq)
        inter = len(retrieved.intersection(gold_set))
        recall_list.append(inter / max(1, len(gold_set)))
    rec = round(float(np.mean(recall_list)), 4) if recall_list else None
    mrr = round(float(np.mean(rr_list)), 4) if rr_list else None
    return rec, mrr

def _dedup_and_rerank(hits: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
    """
    Keep the best-scoring occurrence per UID (or per (id,chunk_id) fallback), then re-rank.
    """
    best: Dict[str, Dict[str, Any]] = {}
    for h in hits:
        uid = h.get("uid")
        if not uid:
            cid = h.get("chunk_id")
            uid = f"{h.get('id','')}::c{int(cid) if cid is not None else -1}"
        prev = best.get(uid)
        if (prev is None) or (float(h.get("score", -1e30)) > float(prev.get("score", -1e30))):
            best[uid] = {**h, "uid": uid}
    # Re-rank by (score desc, uid asc)
    merged = list(best.values())
    merged.sort(key=lambda r: (-float(r.get("score", 0.0)), str(r.get("uid",""))))
    for i, h in enumerate(merged, start=1):
        h["rank"] = i
    return merged

# ---------------- main bench logic ---------------- #

def run_bench(
    config_path: str,
    index_key: str | None,
    backend: str | None,
    qfile: str,
    top_k: int,
    iters: int,
    warmup: int,
    filter_source: str | None,
    out_format: str,
    results_dir: str,
    truth_json: Optional[str] = None,
):
    # Load raw config to determine effective index_key and some knobs for env capture
    with open(config_path, "r", encoding="utf-8") as f:
        raw_cfg = json.load(f)
    effective_index = index_key or raw_cfg.get("active_index")
    if not effective_index:
        raise SystemExit("No index specified and no active_index in config.")
    icfg = raw_cfg["indices"][effective_index]
    cfg_hash = sha1_of_json(icfg)

    # Figure out score semantics (for summary)
    metric = (icfg.get("metric") or "ip").lower()
    # FAISS returns similarity for ip/cosine; Chroma backend converts distance->similarity
    score_type = "similarity" if metric in ("ip", "cosine") else "distance"

    # Identify shard count (if sharded FAISS)
    shard_count = None
    if isinstance(icfg.get("index_paths"), list):
        shard_count = len(icfg.get("index_paths"))

    # Pull common knobs
    ef_search = icfg.get("efSearch")
    extras = icfg.get("extras", {})
    alpha = extras.get("alpha")
    model_name = icfg.get("model_name")

    # Prepare timestamped run dir: results/<index_key>/<YYYYMMDD-HHMMSS>-<cfg_hash>/
    ts = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    run_dir = Path(results_dir) / effective_index / f"{ts}-{cfg_hash}"
    run_dir.mkdir(parents=True, exist_ok=True)
    logs_dir = run_dir / "logs"
    logs_dir.mkdir(parents=True, exist_ok=True)
    log_path = logs_dir / "run.log"

    def log(msg: str):
        print(msg)
        with open(log_path, "a", encoding="utf-8") as fh:
            fh.write(msg + "\n")

    # Save a config snapshot and args for reproducibility
    with open(run_dir / "config_effective.json", "w", encoding="utf-8") as f:
        json.dump(icfg, f, indent=2)
    args_snapshot = {
        "config": config_path,
        "index": index_key,
        "backend": backend,
        "qfile": qfile,
        "top_k": top_k,
        "iters": iters,
        "warmup": warmup,
        "filter_source": filter_source,
        "results_dir": results_dir,
        "truth_json": truth_json,
    }
    with open(run_dir / "args.json", "w", encoding="utf-8") as f:
        json.dump(args_snapshot, f, indent=2)

    # Echo basic setup
    queries = read_queries(qfile)
    n_q = len(queries)
    log(f"[bench] config={config_path} index={effective_index} backend={backend or '(config)'}")
    log(f"[bench] queries={n_q} top_k={top_k} iters={iters} warmup={warmup} filter_source={filter_source or 'None'}")
    log(f"[bench] run_dir={run_dir}")

    # Load pipeline once (measure)
    t_load0 = time.perf_counter()
    P = load_pipeline(config_path, index_key=effective_index, backend_override=backend)
    t_load1 = time.perf_counter()
    load_ms = (t_load1 - t_load0) * 1000.0
    log(f"[bench] pipeline loaded in {load_ms:.1f} ms")

    # Warmup
    if warmup > 0:
        log(f"[bench] warmup passes={warmup}")
        for _ in range(warmup):
            for q in queries:
                _ = P.search_one(q, top_k=top_k, filter_source=filter_source)

    # Measured runs
    per_query_ms: List[float] = []
    all_hits_for_mix: List[Dict[str, Any]] = []
    hits_rows_accum: List[Dict[str, Any]] = []
    hits_by_query_last_iter: Dict[str, List[Dict[str, Any]]] = {}

    t0 = time.perf_counter()
    for it in range(iters):
        log(f"[bench] iter={it+1}/{iters}")
        for q in queries:
            t_q0 = time.perf_counter()
            raw_hits = P.search_one(q, top_k=top_k, filter_source=filter_source)
            # de-dup + re-rank to ensure apples-to-apples
            hits = _dedup_and_rerank(raw_hits)[:top_k]
            t_q1 = time.perf_counter()

            per_query_ms.append((t_q1 - t_q0) * 1000.0)
            all_hits_for_mix.extend(hits)
            hits_by_query_last_iter[q] = hits  # keep last iter (already deduped)
            if out_format in ("csv", "jsonl"):
                hits_rows_accum.extend(flat_hits_rows(q, hits, iter_id=it+1))
    t1 = time.perf_counter()
    P.close()

    total_ms = (t1 - t0) * 1000.0
    total_queries = n_q * iters
    avg_ms = (sum(per_query_ms) / len(per_query_ms)) if per_query_ms else 0.0
    p50_ms = percentile(per_query_ms, 50.0)
    p90_ms = percentile(per_query_ms, 90.0)
    qps = (total_queries / (total_ms / 1000.0)) if total_ms > 0 else 0.0

    # Source mix (normalize)
    mix_counts: Dict[str, int] = {}
    for h in all_hits_for_mix:
        src = (h.get("source") or "").strip().lower() or "unknown"
        mix_counts[src] = mix_counts.get(src, 0) + 1
    total_hits = sum(mix_counts.values()) or 1
    source_mix = {k: round(v / total_hits, 4) for k, v in sorted(mix_counts.items())}

    # Quality probes (optional)
    truth = load_truth_map(truth_json)
    recall_at_k, mrr = compute_recall_mrr(hits_by_query_last_iter, truth) if truth else (None, None)

    # Environment capture
    env = {
        "python": sys.version.split()[0],
        "platform": platform.platform(),
        "numpy": np.__version__,
        "faiss": getattr(faiss, "__version__", "unknown") if _FAISS_OK else "not_installed",
        "faiss_threads": int(getattr(faiss, "omp_get_max_threads", lambda: 0)()) if _FAISS_OK else None,
        "chromadb": _CHROMA_VER,
        "sentence_transformers": _ST_VER,
        "cpu_count": os.cpu_count(),
    }

    summary = {
        "backend": (backend or "config.default"),
        "index_key": effective_index,
        "config_hash": cfg_hash,
        "model_name": model_name,
        "metric": metric,
        "score_type": score_type,                 # similarity|distance
        "efSearch": ef_search,
        "alpha": alpha,
        "shard_count": shard_count,
        "n_queries": n_q,
        "iters": iters,
        "top_k": top_k,
        "avg_ms": round(avg_ms, 2),
        "p50_ms": round(p50_ms, 2),
        "p90_ms": round(p90_ms, 2),
        "qps": round(qps, 2),
        "source_mix": source_mix,
        "filter_source": (filter_source.strip().lower() if filter_source else None),
        "total_elapsed_ms": round(total_ms, 1),
        "quality": {
            "recall_at_k": recall_at_k,
            "mrr": mrr
        },
        "environment": env,
        "run_dir": str(run_dir),
        "hits_written": bool(out_format in ("csv","jsonl")),
    }

    # Write summary to timestamped run_dir
    out_summary_path = run_dir / "bench_summary.json"
    with open(out_summary_path, "w", encoding="utf-8") as f:
        json.dump(summary, f, indent=2)
    log(f"[bench] wrote summary → {out_summary_path}")

    # Optional detailed hits (also under run_dir)
    if out_format == "jsonl":
        out_hits = run_dir / "hits.jsonl"
        with open(out_hits, "w", encoding="utf-8") as f:
            for row in hits_rows_accum:
                f.write(json.dumps(row, ensure_ascii=False) + "\n")
        log(f"[bench] wrote hits jsonl → {out_hits}")
    elif out_format == "csv":
        out_hits = run_dir / "hits.csv"
        if hits_rows_accum:
            fieldnames = list(hits_rows_accum[0].keys())
            with open(out_hits, "w", newline="", encoding="utf-8") as f:
                w = csv.DictWriter(f, fieldnames=fieldnames)
                w.writeheader()
                w.writerows(hits_rows_accum)
        log(f"[bench] wrote hits csv   → {out_hits}")

    # Echo summary
    print(json.dumps(summary, indent=2))


def main():
    ap = argparse.ArgumentParser()
    ap.add_argument("--config", default="configs/index/config.json")
    ap.add_argument("--index", default=None, help="Index key in config (defaults to active_index)")
    ap.add_argument("--backend", default=None, choices=["faiss","chroma"], help="Override backend")
    ap.add_argument("--qfile", required=True, help="Newline-delimited queries file")
    ap.add_argument("--top_k", type=int, default=10)
    ap.add_argument("--iters", type=int, default=1)
    ap.add_argument("--warmup", type=int, default=0)
    ap.add_argument("--filter_source", default=None, help="wikipedia | pubmed | stackexchange")
    ap.add_argument("--format", default="jsonl", choices=["jsonl","csv","none"])
    ap.add_argument("--results_dir", default="results")
    ap.add_argument("--truth_json", default=None, help="Optional JSON with gold uids/ids per query for recall/MRR")
    args = ap.parse_args()

    run_bench(
        config_path=args.config,
        index_key=args.index,
        backend=args.backend,
        qfile=args.qfile,
        top_k=args.top_k,
        iters=args.iters,
        warmup=args.warmup,
        filter_source=args.filter_source,
        out_format=args.format if args.format != "none" else "",
        results_dir=args.results_dir,
        truth_json=args.truth_json,
    )

if __name__ == "__main__":
    main()

Overwriting bench.py


In [ ]:
!python bench.py \
  --config configs/index/config.json \
  --backend faiss \
  --index multi_sharded \
  --qfile data/queries/all_sources_eval.txt \
  --top_k 10 \
  --iters 1 \
  --warmup 0 \
  --format jsonl \
  --results_dir results

E0000 00:00:1764867576.547531   13536 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764867576.554042   13536 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1764867576.570345   13536 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764867576.570378   13536 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764867576.570382   13536 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764867576.570386   13536 computation_placer.cc:177] computation placer already registered. Please check linka

quick sanity

In [ ]:
# Hard-reload pipeline modules
import sys
for m in list(sys.modules):
    if m.startswith("pipeline."):
        del sys.modules[m]

from pipeline.runner import load_pipeline

P = load_pipeline("configs/index/config.json", index_key="multi_sharded", backend_override="faiss")
hits = P.search_one("Explain PageRank algorithm", top_k=5)
print("Sharded OK, first title:", hits[0]["title"], "| source:", hits[0]["source"])
P.close()

[FAISS] Using 12 threads
Sharded OK, first title: Search engine | source: wikipedia


Run benchmarks (FAISS single / sharded / Chroma)

In [ ]:
# FAISS single (no filter)
!python bench.py --config configs/index/config.json --backend faiss --index multi_default \
  --qfile data/queries/all_sources_eval.txt --top_k 10 --iters 3 --warmup 1 --results_dir results

# FAISS sharded (if configured)
!python bench.py --config configs/index/config.json --backend faiss --index multi_sharded \
  --qfile data/queries/all_sources_eval.txt --top_k 10 --iters 3 --warmup 1 --results_dir results

# Chroma
!python bench.py --config configs/index/config.json --backend chroma --index multi_chroma \
  --qfile data/queries/all_sources_eval.txt --top_k 10 --iters 3 --warmup 1 --results_dir results

E0000 00:00:1764867598.848931   13726 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764867598.855359   13726 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1764867598.871340   13726 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764867598.871369   13726 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764867598.871373   13726 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764867598.871375   13726 computation_placer.cc:177] computation placer already registered. Please check linka

Run per-domain slices (to confirm filtering & mix):

In [ ]:
for src in ["wikipedia","pubmed","stackexchange"]:
  !python bench.py --config configs/index/config.json --backend faiss --index multi_default \
    --qfile data/queries/all_sources_eval.txt --top_k 10 --iters 3 --warmup 1 \
    --filter_source {src} --results_dir results

E0000 00:00:1764867653.871291   14150 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764867653.877919   14150 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1764867653.894589   14150 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764867653.894621   14150 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764867653.894624   14150 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764867653.894628   14150 computation_placer.cc:177] computation placer already registered. Please check linka

Sanity peek at what got saved

In [ ]:
import json
from pathlib import Path
import pandas as pd

base = Path("results")
index_key = "multi_default"  # change if you want a different index

# 1) Prefer timestamped run directories (results/<index>/<YYYYMMDD-HHMMSS>-<hash>/)
cand = base / index_key
if not cand.exists():
    raise SystemExit(f"No path: {cand}")

# Filter to directories only
runs = sorted([p for p in cand.iterdir() if p.is_dir()], reverse=True)

if runs:
    run_dir = runs[0]
    print("Inspecting run directory:", run_dir)

    summ_path = run_dir / "bench_summary.json"
    if summ_path.exists():
        summary = json.load(open(summ_path, "r", encoding="utf-8"))
        print(json.dumps(summary, indent=2)[:2000], "...\n")
    else:
        print("No bench_summary.json in this run dir.")

    # Prefer JSONL, else CSV, if present
    hits_jsonl = run_dir / "hits.jsonl"
    hits_csv   = run_dir / "hits.csv"
    if hits_jsonl.exists():
        display(pd.read_json(hits_jsonl, lines=True).head())
    elif hits_csv.exists():
        display(pd.read_csv(hits_csv).head())
    else:
        print("No detailed hits file in this run directory.")

else:
    # 2) No run directories found — maybe you only have a flat CSV/JSONL like all_sources_top10.csv
    print("No run directories under", cand, "— looking for flat files…")

    flat_jsonl = sorted(cand.glob("*.jsonl"), reverse=True)
    flat_csv   = sorted(cand.glob("*.csv"), reverse=True)

    if flat_jsonl:
        f = flat_jsonl[0]
        print("Showing flat JSONL file:", f)
        display(pd.read_json(f, lines=True).head())
    elif flat_csv:
        f = flat_csv[0]
        print("Showing flat CSV file:", f)
        display(pd.read_csv(f).head())
    else:
        raise SystemExit("Found neither run directories nor flat hits files to inspect.")

Inspecting run directory: results/multi_default/20251204-170134-c96953b290c4
{
  "backend": "faiss",
  "index_key": "multi_default",
  "config_hash": "c96953b290c4",
  "model_name": "BAAI/bge-small-en-v1.5",
  "metric": "ip",
  "score_type": "similarity",
  "efSearch": 128,
  "alpha": null,
  "shard_count": null,
  "n_queries": 12,
  "iters": 3,
  "top_k": 10,
  "avg_ms": 12.31,
  "p50_ms": 12.2,
  "p90_ms": 13.4,
  "qps": 79.23,
  "source_mix": {
    "stackexchange": 1.0
  },
  "filter_source": "stackexchange",
  "total_elapsed_ms": 454.4,
  "quality": {
    "recall_at_k": null,
    "mrr": null
  },
  "environment": {
    "python": "3.12.12",
    "platform": "Linux-6.6.105+-x86_64-with-glibc2.35",
    "numpy": "1.26.4",
    "faiss": "1.13.0",
    "faiss_threads": 12,
    "chromadb": "0.4.24",
    "sentence_transformers": "5.1.2",
    "cpu_count": 12
  },
  "run_dir": "results/multi_default/20251204-170134-c96953b290c4",
  "hits_written": true
} ...



,query,iter,rank,score,id,uid,source,title,url,chunk_id,text
0,How to fix ModuleNotFoundError in Python,1,1,0.613627,d7553950-bccc-47a2-acd7-9b389835edfb,d7553950-bccc-47a2-acd7-9b389835edfb::c8,stackexchange,Xwayland gradually consumes all RAM when runni...,https://stackoverflow.com/questions/79750814/x...,8,
1,How to fix ModuleNotFoundError in Python,1,2,0.607274,295167ea-5826-4ba3-b808-f32d2254cda7,295167ea-5826-4ba3-b808-f32d2254cda7::c0,stackexchange,FastAPI won&#39;t recognize module changes,https://stackoverflow.com/questions/79772118/f...,0,
2,How to fix ModuleNotFoundError in Python,1,3,0.605724,c3e108d0-54ec-425f-819e-a3f02b288020,c3e108d0-54ec-425f-819e-a3f02b288020::c1,stackexchange,"Error while deploying, but not in local: &quot...",https://stackoverflow.com/questions/79782221/e...,1,
3,How to fix ModuleNotFoundError in Python,1,4,0.605515,66fbaa8b-6915-4ddd-a731-08566066975e,66fbaa8b-6915-4ddd-a731-08566066975e::c2,stackexchange,No module named &#39;wx._msw&#39;,https://stackoverflow.com/questions/79784586/n...,2,
4,How to fix ModuleNotFoundError in Python,1,5,0.600347,c1067325-5e06-416a-b590-33b94c647bc6,c1067325-5e06-416a-b590-33b94c647bc6::c0,stackexchange,adding hidden imports in .spec file doesnt see...,https://stackoverflow.com/questions/79757171/a...,0,


## **Step 3.6: Spark fan-out skeleton**

In [ ]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"    # 0=all, 1=filter INFO, 2=filter WARNING, 3=filter ERROR
os.environ["XLA_FLAGS"] = "--xla_gpu_cuda_data_dir=/usr/local/cuda"
# Also keep Spark quiet:
os.environ["PYSPARK_SUBMIT_ARGS"] = "--conf spark.ui.showConsoleProgress=false pyspark-shell"

In [ ]:
# --- Colab Spark + Java setup ---
!apt -qq install openjdk-17-jdk-headless -y  # Java 17 (good for Spark 3.5+)
import os

# set env vars
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-17-openjdk-amd64"
os.environ["SPARK_HOME"] = "/usr/local/lib/python3.12/dist-packages/pyspark"
os.environ["PATH"] += f":{os.environ['SPARK_HOME']}/bin:{os.environ['JAVA_HOME']}/bin"

# optional silencing
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
os.environ["TOKENIZERS_PARALLELISM"] = "false"

# verify
!java -version
!echo $JAVA_HOME

openjdk-17-jdk-headless is already the newest version (17.0.16+8~us1-0ubuntu1~22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.
openjdk version "17.0.16" 2025-07-15
OpenJDK Runtime Environment (build 17.0.16+8-Ubuntu-0ubuntu122.04.1)
OpenJDK 64-Bit Server VM (build 17.0.16+8-Ubuntu-0ubuntu122.04.1, mixed mode, sharing)
/usr/lib/jvm/java-17-openjdk-amd64


In [ ]:
%%writefile scripts/spark_fanout.py
import os, json, time, argparse, math
from typing import List, Tuple, Dict, Any, Iterator

# Quieter HuggingFace tokenizers in forked workers
os.environ.setdefault("TOKENIZERS_PARALLELISM", "false")

import numpy as np
import pandas as pd

from pyspark.sql import SparkSession
from sentence_transformers import SentenceTransformer

try:
    import faiss  # faiss-cpu
except Exception:
    import faiss_cpu as faiss  # fallback name if env aliases it


def _set_efsearch(index, ef: int) -> None:
    try:
        ps = faiss.ParameterSpace()
        ps.set_index_parameter(index, "efSearch", int(ef))
    except Exception:
        pass


def _encode_queries(qfile: str, model_name: str, normalize: bool) -> Tuple[List[str], np.ndarray]:
    queries: List[str] = []
    with open(qfile, "r", encoding="utf-8") as f:
        for ln in f:
            ln = ln.strip()
            if ln:
                queries.append(ln)
    if not queries:
        raise SystemExit(f"No queries found in {qfile}")

    model = SentenceTransformer(model_name)
    qvecs = model.encode(queries, normalize_embeddings=normalize)
    qvecs = np.asarray(qvecs, dtype="float32")
    return queries, qvecs


def _row_to_rd(row: pd.Series) -> Dict[str, Any]:
    src_raw = row.get("domain", "") or row.get("source", "")
    src = str(src_raw).strip().lower()
    cid = row.get("chunk_id", None)
    try:
        cid = int(cid) if cid is not None and cid == cid else None
    except Exception:
        cid = None
    uid = f"{row.get('id','')}::c{cid if cid is not None else -1}"
    text = (row.get("chunk_text", "") or row.get("text", "") or row.get("preview", "") or "")
    return {
        "id": str(row.get("id", "")),
        "uid": uid,
        "source": src,
        "title": str(row.get("title", "")),
        "url": str(row.get("url", "")),
        "text": str(text),
        "chunk_id": cid,
    }


def _search_shard_for_batch(
    ipath: str,
    mpath: str,
    qvecs_b: np.ndarray,
    top_k_prime: int,
    efsearch: int,
    filter_source: str | None,
) -> List[Tuple[int, float, Dict[str, Any]]]:
    """
    Search one shard for all query vectors in the broadcast batch.
    Returns flat (qidx, score, rowdict) tuples.
    """
    index = faiss.read_index(ipath)
    _set_efsearch(index, efsearch)
    meta = pd.read_parquet(mpath).reset_index(drop=True)

    out: List[Tuple[int, float, Dict[str, Any]]] = []

    # Vectorized search per shard (one FAISS call per qvec)
    for qi in range(qvecs_b.shape[0]):
        qv = qvecs_b[qi:qi+1]
        D, I = index.search(qv, top_k_prime)
        for score, li in zip(D[0], I[0]):
            if li < 0:
                continue
            row = meta.iloc[int(li)]
            src_raw  = row.get("domain", "") or row.get("source", "") or ""
            src_norm = str(src_raw).strip().lower()
            if filter_source and src_norm != filter_source.strip().lower():
                continue
            out.append((qi, float(score), _row_to_rd(row)))

    return out


def _map_partitions_search(
    it: Iterator[Tuple[str, str]],
    qvecs_b: np.ndarray,
    top_k_prime: int,
    efsearch: int,
    filter_source: str | None,
) -> Iterator[List[Tuple[int, float, Dict[str, Any]]]]:
    """
    mapPartitions: each partition receives one or more (index_path, meta_path)
    tuples; for each shard in the partition, load index ONCE and search all queries.
    """
    results_all: List[Tuple[int, float, Dict[str, Any]]] = []
    for ipath, mpath in it:
        part_out = _search_shard_for_batch(
            ipath=ipath,
            mpath=mpath,
            qvecs_b=qvecs_b,
            top_k_prime=top_k_prime,
            efsearch=efsearch,
            filter_source=filter_source,
        )
        results_all.extend(part_out)
    yield results_all


def main():
    ap = argparse.ArgumentParser()
    ap.add_argument("--manifest", required=True,
                    help="Path to indices/multi_default_sharded/manifest.json")
    ap.add_argument("--qfile", required=True, help="Newline-delimited queries file")
    ap.add_argument("--model", default="BAAI/bge-small-en-v1.5")
    ap.add_argument("--normalize_query", type=int, default=1)
    ap.add_argument("--top_k", type=int, default=10)
    ap.add_argument("--alpha", type=int, default=3, help="overfetch factor per shard")
    ap.add_argument("--efsearch", type=int, default=128)
    ap.add_argument("--filter_source", default=None, help="wikipedia|pubmed|stackexchange")
    ap.add_argument("--results_csv", default="results/spark_fanout_hits.csv")
    ap.add_argument("--log", default="logs/spark_fanout_test.log")
    args = ap.parse_args()

    os.makedirs(os.path.dirname(args.results_csv), exist_ok=True)
    os.makedirs(os.path.dirname(args.log), exist_ok=True)

    # ---- Load manifest
    man = json.load(open(args.manifest))
    # Support both "index_paths/meta_paths" and richer shard list
    if "shards" in man and isinstance(man["shards"], list):
        index_paths = [s["index_path"] for s in man["shards"]]
        meta_paths  = [s["meta_path"]  for s in man["shards"]]
        metric      = (man.get("metric") or man.get("index_metric") or "ip").lower()
        normalized  = bool(man.get("normalized", True))
    else:
        index_paths = man.get("index_paths") or []
        meta_paths  = man.get("meta_paths") or []
        metric      = (man.get("metric") or man.get("index_metric") or "ip").lower()
        normalized  = bool(man.get("normalized", True))

    if not index_paths or not meta_paths or len(index_paths) != len(meta_paths):
        raise SystemExit("Manifest missing or paths length mismatch.")

    # ---- Metric parity & normalization checks (fail-fast)
    if metric not in {"ip", "cosine"}:
        # If shards were built as L2, we can still work but ranking semantics change (lower=better).
        # For fan-out we assume IP/cosine (higher=better). Enforce now to keep heap logic consistent.
        raise SystemExit(
            f"Manifest reports metric='{metric}'. Expected 'ip' or 'cosine'. "
            f"Rebuild shards with IP on normalized vectors (see Step 3.3)."
        )
    if not normalized:
        # IP≈cosine requires normalized corpus + (usually) normalized queries
        raise SystemExit(
            "Manifest indicates 'normalized=false'. Rebuild corpus vectors as unit-norm for IP/cosine parity."
        )

    shards = list(zip(index_paths, meta_paths))

    # ---- Encode on driver (broadcast vectors, not the model)
    t0 = time.perf_counter()
    queries, qvecs = _encode_queries(args.qfile, args.model, bool(args.normalize_query))
    t1 = time.perf_counter()

    # ---- Spark session (local; disable Arrow—FAISS is native)
    spark = (
        SparkSession.builder
        .appName("spark_fanout_local")
        .master("local[*]")
        .config("spark.ui.showConsoleProgress", "false")
        .config("spark.sql.execution.arrow.pyspark.enabled", "false")
        .getOrCreate()
    )
    spark.sparkContext.setLogLevel("ERROR")
    sc = spark.sparkContext

    # ---- Broadcast query vectors and log broadcast size
    qvecs_b = sc.broadcast(qvecs)
    est_bcast_bytes = qvecs.nbytes
    # Simple size guard: if > 50MB suggest batch-splitting
    bcast_warn = (est_bcast_bytes > 50 * 1024 * 1024)

    per_shard = max(args.top_k * args.alpha, args.top_k)

    # ---- Map each shard → top-k' per query using mapPartitions (load index once per shard)
    t2 = time.perf_counter()
    # One shard per partition for clean locality
    rdd = sc.parallelize(shards, len(shards))
    shard_results_nested = rdd.mapPartitions(
        lambda it: _map_partitions_search(
            it=it,
            qvecs_b=qvecs_b.value,
            top_k_prime=per_shard,
            efsearch=args.efsearch,
            filter_source=args.filter_source,
        )
    ).collect()
    t3 = time.perf_counter()

    # ---- Merge to global top-k per query, deterministic tie-break (score desc, uid asc)
    flat: List[Tuple[int, float, Dict[str, Any]]] = []
    for part in shard_results_nested:
        flat.extend(part)

    # Dedup by uid with best score kept
    by_q: Dict[int, Dict[str, Tuple[float, Dict[str, Any]]]] = {}
    for qi, score, rd in flat:
        bucket = by_q.setdefault(qi, {})
        uid = rd["uid"]
        if (uid not in bucket) or (score > bucket[uid][0]):
            bucket[uid] = (score, rd)

    rows = []
    for qi, uid_map in by_q.items():
        # stable key: (score desc, uid asc)
        candidates = [(score, rd["uid"], rd) for uid, (score, rd) in uid_map.items()]
        candidates.sort(key=lambda x: (-x[0], x[1]))
        top = candidates[: args.top_k]
        for rank, (score, _uid, rd) in enumerate(top, start=1):
            rows.append({
                "query": queries[qi],
                "rank": rank,
                "score": float(score),
                "id": rd["id"],
                "uid": rd["uid"],
                "source": rd["source"],
                "title": rd["title"],
                "url": rd["url"],
                "chunk_id": rd["chunk_id"],
                "text": rd["text"],
            })

    rows.sort(key=lambda r: (r["query"], r["rank"]))
    df = pd.DataFrame(rows)
    os.makedirs(os.path.dirname(args.results_csv), exist_ok=True)
    df.to_csv(args.results_csv, index=False)

    # ---- timings & log
    t4 = time.perf_counter()
    timings = {
        "n_shards": len(shards),
        "n_queries": len(queries),
        "top_k": args.top_k,
        "alpha": args.alpha,
        "efsearch": args.efsearch,
        "encode_ms": round((t1 - t0) * 1000, 1),
        "spark_map_ms": round((t3 - t2) * 1000, 1),
        "merge_ms": round((t4 - t3) * 1000, 1),
        "total_ms": round((t4 - t0) * 1000, 1),
        "filter_source": args.filter_source or None,
        "results_csv": args.results_csv,
        "metric": metric,
        "normalized": bool(args.normalize_query),
        "broadcast_bytes": est_bcast_bytes,
        "broadcast_large_hint": bcast_warn,
    }
    os.makedirs(os.path.dirname(args.log), exist_ok=True)
    with open(args.log, "w", encoding="utf-8") as fh:
        fh.write(json.dumps(timings, indent=2) + "\n")

    print("=== Spark fan-out summary ===")
    print(json.dumps(timings, indent=2))

    spark.stop()


if __name__ == "__main__":
    main()

Overwriting scripts/spark_fanout.py


Auto-rebuild it from your shard folders

In [ ]:
# 🔧 Rebuild manifest from existing shard files (run in project root)
import os, glob, json

ROOT = os.getcwd()  # should be "/content/drive/MyDrive/AMS 560 PROJECT"
idx_glob  = os.path.join(ROOT, "indices/multi_default_sharded/shard_*.faiss")
meta_glob = os.path.join(ROOT, "data/embeddings/multi_default_sharded/meta_shard_*.parquet")

index_paths = sorted(glob.glob(idx_glob))
meta_paths  = sorted(glob.glob(meta_glob))

print("#index_paths:", len(index_paths))
print("#meta_paths :", len(meta_paths))
print("Sample index:", index_paths[:2])
print("Sample meta :", meta_paths[:2])

if not index_paths or not meta_paths or len(index_paths) != len(meta_paths):
    raise SystemExit("Shard discovery failed — check that both shard .faiss and meta_shard_*.parquet exist.")

# If you know your shards were built with IP on normalized vectors (Step 3.3 fix):
metric = "ip"
normalized = True
M = 32
efConstruction = 200

man = {
    "index_paths": index_paths,
    "meta_paths": meta_paths,
    "metric": metric,
    "normalized": normalized,
    "M": M,
    "efConstruction": efConstruction,
    # (optional richer format that scripts/spark_fanout.py can also read)
    "shards": [
        {
            "id": f"{i:03d}",
            "index_path": ip,
            "meta_path": mp,
            "n": None  # can be filled later if you want
        }
        for i, (ip, mp) in enumerate(zip(index_paths, meta_paths))
    ]
}
man_path = os.path.join(ROOT, "indices/multi_default_sharded/manifest.json")
os.makedirs(os.path.dirname(man_path), exist_ok=True)
with open(man_path, "w") as f:
    json.dump(man, f, indent=2)
print("✅ Wrote manifest:", man_path)

#index_paths: 8
#meta_paths : 8
Sample index: ['/content/drive/MyDrive/AMS 560 PROJECT/indices/multi_default_sharded/shard_000.faiss', '/content/drive/MyDrive/AMS 560 PROJECT/indices/multi_default_sharded/shard_001.faiss']
Sample meta : ['/content/drive/MyDrive/AMS 560 PROJECT/data/embeddings/multi_default_sharded/meta_shard_000.parquet', '/content/drive/MyDrive/AMS 560 PROJECT/data/embeddings/multi_default_sharded/meta_shard_001.parquet']
✅ Wrote manifest: /content/drive/MyDrive/AMS 560 PROJECT/indices/multi_default_sharded/manifest.json


run the script

In [ ]:
MANIFEST = "indices/multi_default_sharded/manifest.json"   # or absolute path
QFILE    = "data/queries/all_sources_eval.txt"

!python scripts/spark_fanout.py \
  --manifest "$MANIFEST" \
  --qfile "$QFILE" \
  --top_k 10 \
  --alpha 1 \
  --efsearch 128 \
  --normalize_query 1 \
  --results_csv "results/spark_fanout_hits.csv" \
  --log "logs/spark_fanout_test.log"

2025-12-04 17:28:20.562872: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764869300.584720   21472 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764869300.591311   21472 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1764869300.607723   21472 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764869300.607749   21472 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764869300.607752   21472 computation_placer.cc:177] computation placer alr

**Verify Spark top-k equals your local sharded runner's top-k**

In [ ]:
glob.glob("results/*.csv")
['results/spark_fanout_hits.csv']


['results/spark_fanout_hits.csv']

In [ ]:
%cd "/content/drive/MyDrive/AMS 560 PROJECT"


/content/drive/MyDrive/AMS 560 PROJECT


In [ ]:
import json

CFG = "configs/index/config.json"

with open(CFG, "r") as f:
    cfg = json.load(f)

print("Available indices keys:")
print(list(cfg.get("indices", {}).keys()))

print("\nactive_index in config:")
print(cfg.get("active_index"))


Available indices keys:
['multi_default', 'multi_sharded', 'multi_chroma']

active_index in config:
multi_chroma


In [ ]:
import os
import pandas as pd
from pipeline.runner import load_pipeline

CFG   = "configs/index/config.json"
QFILE = "data/queries/all_sources_eval.txt"

# Load the FAISS sharded index locally (same one you used to build shards)
P = load_pipeline(CFG, index_key="multi_sharded")

# Read queries
with open(QFILE, "r", encoding="utf-8") as f:
    queries = [ln.strip() for ln in f if ln.strip()]

rows = []

for q in queries:
    hits = P.search_one(q, top_k=10)   # list of dicts
    for rank, h in enumerate(hits, start=1):
        # h is a dict, so use h[...] / h.get(...)
        chunk_id = h.get("chunk_id", None)
        uid = h.get("uid", None)

        if uid is None:
            # robust cid handling
            if chunk_id is None or (isinstance(chunk_id, float) and pd.isna(chunk_id)):
                cid = -1
            else:
                cid = int(chunk_id)
            uid = f"{h['id']}::c{cid}"

        rows.append({
            "query": q,
            "rank": rank,
            "score": float(h.get("score", 0.0)),
            "id": h.get("id", ""),
            "uid": uid,
            "source": h.get("source"),
            "title": h.get("title"),
            "url": h.get("url"),
            "chunk_id": chunk_id,
            "iter": 1,   # single baseline run
        })

df_local = pd.DataFrame(rows)

os.makedirs("results", exist_ok=True)
out_path = "results/local_multi_default_sharded_runs.csv"
df_local.to_csv(out_path, index=False)

print("Wrote local baseline to:", out_path)
print("Shape:", df_local.shape)
df_local.head()


[FAISS] Using 12 threads
Wrote local baseline to: results/local_multi_default_sharded_runs.csv
Shape: (120, 10)


,query,rank,score,id,uid,source,title,url,chunk_id,iter
0,metformin mechanism of action,1,0.736435,879c2390-c660-4fe3-8a31-f959c9c013c4,879c2390-c660-4fe3-8a31-f959c9c013c4::c0,pubmed,The beneficial effects of empagliflozin-metfor...,https://pubmed.ncbi.nlm.nih.gov/41175191/,0,1
1,metformin mechanism of action,2,0.734521,ac6d737e-edaa-4421-929c-969f67d09e52,ac6d737e-edaa-4421-929c-969f67d09e52::c0,pubmed,Randomized controlled trial of effects of metf...,https://pubmed.ncbi.nlm.nih.gov/41174662/,0,1
2,metformin mechanism of action,3,0.728445,0e2a1ab5-3c8f-4ffb-80f2-7f3892ea7e5e,0e2a1ab5-3c8f-4ffb-80f2-7f3892ea7e5e::c0,pubmed,Computational Modelling the Impact of GLP-1 Re...,https://pubmed.ncbi.nlm.nih.gov/41173332/,0,1
3,metformin mechanism of action,4,0.724863,14d305f3-b590-4ed4-9bba-f14800f2c40f,14d305f3-b590-4ed4-9bba-f14800f2c40f::c0,pubmed,Metformin induces ferroptosis associated with ...,https://pubmed.ncbi.nlm.nih.gov/41172231/,0,1
4,metformin mechanism of action,5,0.716767,a2fb6d19-85af-4153-be89-e56bb9be99c1,a2fb6d19-85af-4153-be89-e56bb9be99c1::c0,pubmed,Curcumin ameliorates high-glucose-induced lipi...,https://pubmed.ncbi.nlm.nih.gov/41172806/,0,1


In [ ]:
import pandas as pd

df_spark = pd.read_csv("results/spark_fanout_hits.csv")
df_local = pd.read_csv("results/local_multi_default_sharded_runs.csv")

print("Spark fanout shape:", df_spark.shape)
print("Local baseline  shape:", df_local.shape)

df_local_run = df_local[df_local["iter"] == 1].copy()


Spark fanout shape: (120, 10)
Local baseline  shape: (120, 10)


In [ ]:
def mk_uid(df: pd.DataFrame) -> pd.Series:
    if "uid" in df.columns and df["uid"].notna().all():
        return df["uid"].astype(str)
    cid = df.get("chunk_id")
    return df["id"].astype(str) + "::c" + (
        cid.fillna(-1).astype(int).astype(str) if cid is not None else "-1"
    )

df_spark = df_spark.assign(uid=mk_uid(df_spark))
df_local_run = df_local_run.assign(uid=mk_uid(df_local_run))

In [ ]:
def cmp_sets(query: str, k: int = 10):
    a = df_spark[df_spark["query"] == query].sort_values("rank").head(k)["uid"]
    b = df_local_run[df_local_run["query"] == query].sort_values("rank").head(k)["uid"]
    aset, bset = set(a), set(b)
    return {
        "query": query,
        "topk_equal_uid_set": (aset == bset),
        "spark_only": sorted(list(aset - bset))[:5],
        "local_only": sorted(list(bset - aset))[:5],
        "k_compared": min(len(a), len(b)),
        "spark_dupes": a.duplicated().sum(),
        "local_dupes": b.duplicated().sum(),
    }

queries = sorted(set(df_spark["query"]) & set(df_local_run["query"]))
reports = [cmp_sets(q, k=10) for q in queries]
mismatch = [r for r in reports if not r["topk_equal_uid_set"]]

print(f"Queries compared: {len(reports)}")
print(f"Exact top-k UID set match: {len(reports)-len(mismatch)}/{len(reports)}")

if mismatch:
    import pprint
    pprint.pp(mismatch[:3])


Queries compared: 12
Exact top-k UID set match: 12/12


In [ ]:
import json, pandas as pd, pyarrow.parquet as pq
from pathlib import Path

MAN="indices/multi_default_sharded/manifest.json"
m=json.load(open(MAN))
ipaths=m["index_paths"]; mpaths=m["meta_paths"]
print("Shards:", len(ipaths), len(mpaths))

def uidify(df):
    ch = (df["chunk_id"].fillna(-1).astype("int64").astype(str)) if "chunk_id" in df else pd.Series(["row"+str(i) for i in range(len(df))])
    return df["id"].astype(str) + "::c" + ch

uids_all=[]
for i,mp in enumerate(mpaths):
    df=pq.read_table(mp).to_pandas()
    uids=uidify(df)
    print(f"shard {i}: rows={len(df)}, unique_uids={uids.nunique()}")
    uids_all.append(pd.Series(uids.values, name=f"s{i}"))

all_uids=pd.concat(uids_all, ignore_index=True)
print("GLOBAL total rows:", len(all_uids), "global unique UIDs:", all_uids.nunique())
dups = all_uids.value_counts()
print("UIDs appearing in >1 shard:", (dups>1).sum())

Shards: 8 8
shard 0: rows=10184, unique_uids=10184
shard 1: rows=10130, unique_uids=10130
shard 2: rows=9622, unique_uids=9622
shard 3: rows=9362, unique_uids=9362
shard 4: rows=9666, unique_uids=9666
shard 5: rows=10011, unique_uids=10011
shard 6: rows=10331, unique_uids=10331
shard 7: rows=9858, unique_uids=9858
GLOBAL total rows: 79164 global unique UIDs: 79164
UIDs appearing in >1 shard: 0


### **LIVE DEMO**

Step 1: Run a query in FAISS Single

In [ ]:
from pipeline.runner import load_pipeline

P = load_pipeline("configs/index/config.json", index_key="multi_default")
P.search_one("Explain PageRank algorithm", top_k=5)

[FAISS] Using 12 threads


[{'rank': 1,
  'score': 0.5379228591918945,
  'id': 'f52fac4b-452a-4449-8cb1-bae763c4f528',
  'uid': 'f52fac4b-452a-4449-8cb1-bae763c4f528::c13',
  'source': 'wikipedia',
  'title': 'Search engine',
  'url': 'https://en.wikipedia.org/wiki/Search%20engine',
  'text': '',
  'chunk_id': 13},
 {'rank': 2,
  'score': 0.5440587401390076,
  'id': 'f52fac4b-452a-4449-8cb1-bae763c4f528',
  'uid': 'f52fac4b-452a-4449-8cb1-bae763c4f528::c34',
  'source': 'wikipedia',
  'title': 'Search engine',
  'url': 'https://en.wikipedia.org/wiki/Search%20engine',
  'text': '',
  'chunk_id': 34},
 {'rank': 3,
  'score': 0.5530006885528564,
  'id': 'f52fac4b-452a-4449-8cb1-bae763c4f528',
  'uid': 'f52fac4b-452a-4449-8cb1-bae763c4f528::c32',
  'source': 'wikipedia',
  'title': 'Search engine',
  'url': 'https://en.wikipedia.org/wiki/Search%20engine',
  'text': '',
  'chunk_id': 32},
 {'rank': 4,
  'score': 0.5544021129608154,
  'id': 'f52fac4b-452a-4449-8cb1-bae763c4f528',
  'uid': 'f52fac4b-452a-4449-8cb1-bae7

Step 2: Switch to FAISS Sharded

In [ ]:
P = load_pipeline("configs/index/config.json", index_key="multi_sharded")
P.search_one("Explain PageRank algorithm", top_k=5)

[FAISS] Using 12 threads


[{'rank': 1,
  'score': 0.7310386300086975,
  'id': 'f52fac4b-452a-4449-8cb1-bae763c4f528',
  'uid': 'f52fac4b-452a-4449-8cb1-bae763c4f528::c13',
  'source': 'wikipedia',
  'title': 'Search engine',
  'url': 'https://en.wikipedia.org/wiki/Search%20engine',
  'text': '',
  'chunk_id': 13},
 {'rank': 2,
  'score': 0.7279707193374634,
  'id': 'f52fac4b-452a-4449-8cb1-bae763c4f528',
  'uid': 'f52fac4b-452a-4449-8cb1-bae763c4f528::c34',
  'source': 'wikipedia',
  'title': 'Search engine',
  'url': 'https://en.wikipedia.org/wiki/Search%20engine',
  'text': '',
  'chunk_id': 34},
 {'rank': 3,
  'score': 0.7234997153282166,
  'id': 'f52fac4b-452a-4449-8cb1-bae763c4f528',
  'uid': 'f52fac4b-452a-4449-8cb1-bae763c4f528::c32',
  'source': 'wikipedia',
  'title': 'Search engine',
  'url': 'https://en.wikipedia.org/wiki/Search%20engine',
  'text': '',
  'chunk_id': 32},
 {'rank': 4,
  'score': 0.7227989435195923,
  'id': 'f52fac4b-452a-4449-8cb1-bae763c4f528',
  'uid': 'f52fac4b-452a-4449-8cb1-bae7

Step 3: Switch to Chroma

In [ ]:
P = load_pipeline("configs/index/config.json", index_key="multi_chroma")
P.search_one("Explain PageRank algorithm", top_k=5)

ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given


Loaded Chroma collection: multi_default | count=79164 | store=indices/chroma_store


[{'rank': 1,
  'score': 0.46207714080810547,
  'id': 'f52fac4b-452a-4449-8cb1-bae763c4f528',
  'uid': 'f52fac4b-452a-4449-8cb1-bae763c4f528::c13',
  'source': 'wikipedia',
  'title': 'Search engine',
  'url': 'https://en.wikipedia.org/wiki/Search%20engine',
  'text': '',
  'chunk_id': 13},
 {'rank': 2,
  'score': 0.45594125986099243,
  'id': 'f52fac4b-452a-4449-8cb1-bae763c4f528',
  'uid': 'f52fac4b-452a-4449-8cb1-bae763c4f528::c34',
  'source': 'wikipedia',
  'title': 'Search engine',
  'url': 'https://en.wikipedia.org/wiki/Search%20engine',
  'text': '',
  'chunk_id': 34},
 {'rank': 3,
  'score': 0.44699931144714355,
  'id': 'f52fac4b-452a-4449-8cb1-bae763c4f528',
  'uid': 'f52fac4b-452a-4449-8cb1-bae763c4f528::c32',
  'source': 'wikipedia',
  'title': 'Search engine',
  'url': 'https://en.wikipedia.org/wiki/Search%20engine',
  'text': '',
  'chunk_id': 32},
 {'rank': 4,
  'score': 0.44559788703918457,
  'id': 'f52fac4b-452a-4449-8cb1-bae763c4f528',
  'uid': 'f52fac4b-452a-4449-8cb1-

Multi-Domain Hit Differences: Without any domain filter

In [ ]:
P.search_one("side effects of metformin", top_k=5)

[{'rank': 1,
  'score': 0.5025807619094849,
  'id': '2e01f87d-d8f2-4495-8d94-3fe69606e423',
  'uid': '2e01f87d-d8f2-4495-8d94-3fe69606e423::c12',
  'source': 'wikipedia',
  'title': 'Amphetamine',
  'url': 'https://en.wikipedia.org/wiki/Amphetamine',
  'text': '',
  'chunk_id': 12},
 {'rank': 2,
  'score': 0.4798773527145386,
  'id': '2e01f87d-d8f2-4495-8d94-3fe69606e423',
  'uid': '2e01f87d-d8f2-4495-8d94-3fe69606e423::c11',
  'source': 'wikipedia',
  'title': 'Amphetamine',
  'url': 'https://en.wikipedia.org/wiki/Amphetamine',
  'text': '',
  'chunk_id': 11},
 {'rank': 3,
  'score': 0.4463244080543518,
  'id': '8ec8afff-470d-4c86-979d-d8cccb358cb6',
  'uid': '8ec8afff-470d-4c86-979d-d8cccb358cb6::c24',
  'source': 'wikipedia',
  'title': 'Antipsychotic',
  'url': 'https://en.wikipedia.org/wiki/Antipsychotic',
  'text': '',
  'chunk_id': 24},
 {'rank': 4,
  'score': 0.44114363193511963,
  'id': '8ec8afff-470d-4c86-979d-d8cccb358cb6',
  'uid': '8ec8afff-470d-4c86-979d-d8cccb358cb6::c22

applying filters

In [ ]:
P.search_one("side effects of metformin", top_k=5, filter_source="pubmed")

[{'rank': 1,
  'score': 0.4757671356201172,
  'id': '879c2390-c660-4fe3-8a31-f959c9c013c4',
  'uid': '879c2390-c660-4fe3-8a31-f959c9c013c4::c0',
  'source': 'pubmed',
  'title': 'The beneficial effects of empagliflozin-metformin combination on cardiovascular risk factors in type 2 diabetes mellitus patients.',
  'url': 'https://pubmed.ncbi.nlm.nih.gov/41175191/',
  'text': '',
  'chunk_id': 0},
 {'rank': 2,
  'score': 0.469219446182251,
  'id': 'ac6d737e-edaa-4421-929c-969f67d09e52',
  'uid': 'ac6d737e-edaa-4421-929c-969f67d09e52::c0',
  'source': 'pubmed',
  'title': 'Randomized controlled trial of effects of metformin in NAFLD patients with newly diagnosed type 2 diabetes treated with an intensive lifestyle: a study protocol.',
  'url': 'https://pubmed.ncbi.nlm.nih.gov/41174662/',
  'text': '',
  'chunk_id': 0},
 {'rank': 3,
  'score': 0.46206068992614746,
  'id': '8326cb28-4d8e-441a-a139-7e8b72fb09e6',
  'uid': '8326cb28-4d8e-441a-a139-7e8b72fb09e6::c0',
  'source': 'pubmed',
  'tit

Spark Fan-Out Results

In [ ]:
pd.read_csv("results/spark_fanout_hits.csv")

,query,rank,score,id,uid,source,title,url,chunk_id,text
0,Explain PageRank algorithm,1,0.731039,f52fac4b-452a-4449-8cb1-bae763c4f528,f52fac4b-452a-4449-8cb1-bae763c4f528::c13,wikipedia,Search engine,https://en.wikipedia.org/wiki/Search%20engine,13,NaN
1,Explain PageRank algorithm,2,0.727971,f52fac4b-452a-4449-8cb1-bae763c4f528,f52fac4b-452a-4449-8cb1-bae763c4f528::c34,wikipedia,Search engine,https://en.wikipedia.org/wiki/Search%20engine,34,NaN
2,Explain PageRank algorithm,3,0.723500,f52fac4b-452a-4449-8cb1-bae763c4f528,f52fac4b-452a-4449-8cb1-bae763c4f528::c32,wikipedia,Search engine,https://en.wikipedia.org/wiki/Search%20engine,32,NaN
3,Explain PageRank algorithm,4,0.722799,f52fac4b-452a-4449-8cb1-bae763c4f528,f52fac4b-452a-4449-8cb1-bae763c4f528::c11,wikipedia,Search engine,https://en.wikipedia.org/wiki/Search%20engine,11,NaN
4,Explain PageRank algorithm,5,0.697764,0d986bd6-5034-4824-abbc-054048f6ca1f,0d986bd6-5034-4824-abbc-054048f6ca1f::c0,pubmed,A comprehensive framework for solution space e...,https://pubmed.ncbi.nlm.nih.gov/41173948/,0,NaN
...,...,...,...,...,...,...,...,...,...,...
115,type 2 diabetes lifestyle interventions evidence,6,0.779740,18cd8804-db4f-4197-b1e5-b0413a06a388,18cd8804-db4f-4197-b1e5-b0413a06a388::c19,wikipedia,Coronary artery disease,https://en.wikipedia.org/wiki/Coronary%20arter...,19,NaN
116,type 2 diabetes lifestyle interventions evidence,7,0.774791,2cba85e0-6ba6-4841-a397-b1d43b8482a2,2cba85e0-6ba6-4841-a397-b1d43b8482a2::c0,pubmed,Continuous Glucose Monitoring Frequency and Gl...,https://pubmed.ncbi.nlm.nih.gov/41171276/,0,NaN
117,type 2 diabetes lifestyle interventions evidence,8,0.764265,7ec43062-6d7a-460b-89af-5affa29e72b3,7ec43062-6d7a-460b-89af-5affa29e72b3::c0,pubmed,A Scoping Review of Glucose Spikes in People W...,https://pubmed.ncbi.nlm.nih.gov/41170150/,0,NaN
118,type 2 diabetes lifestyle interventions evidence,9,0.763914,6e738fde-2c11-4eda-ba76-284f847dc516,6e738fde-2c11-4eda-ba76-284f847dc516::c0,pubmed,Comprehensive impact of Intermittent Hypoxia T...,https://pubmed.ncbi.nlm.nih.gov/41170361/,0,NaN
